In [ ]:
%tensorflow_version 1.x
!pip install h5py==2.10.0
!pip install git+https://www.github.com/keras-team/keras-contrib.git
!pip install seqeval
!pip install flair
import pandas as pd
import numpy as np
import os
import keras 
from math import nan 
from keras.callbacks import ModelCheckpoint
from keras_contrib.layers import CRF
from future.utils import iteritems
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical # tensor flow vesion 1 
from keras.models import Model, Input
import tensorflow as tf
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report
from keras.initializers import RandomUniform
from keras.layers import TimeDistributed,Conv1D,Conv2D,Dense,Embedding,Input,Dropout,LSTM,Bidirectional,MaxPooling1D,Flatten,concatenate,MaxPooling2D
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
#!pip install keras-self-attention
#from keras_self_attention import SeqSelfAttention
from keras.layers.merge import Concatenate
import flair.datasets
from flair.data import Sentence
from flair.embeddings import WordEmbeddings,TransformerWordEmbeddings,StackedEmbeddings, CharacterEmbeddings, FlairEmbeddings
import sklearn 
from sklearn.model_selection import train_test_split
import json
from keras.models import Model, model_from_json


TensorFlow 1.x selected.
     |████████████████████████████████| 2.9 MB 15.6 MB/s 
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-3192zsbd
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-3192zsbd
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101077 sha256=7a232d9c823caa26626349d68c55393e7f84dc425d3a12f8a984df3cf610d964
  Stored in directory: /tmp/pip-ephem-wheel-cache-xvanc3rq/wheels/bb/1f/f2/b57495012683b6b20bbae94a3915ec79753111452d79886abc
Successfully built keras-contrib
     |████████████████████████████████| 43 kB 1.5 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=87e9e45185c8e5e4cd1f1ba4ac5f920b4b3c281b0267d02067891ec39f18649e
  Stored in directory: /root/.cache/p

Using TensorFlow backend.


In [ ]:
print("keras version: ",keras.__version__)
print("tensorflow version: ",tf.__version__)

keras version:  2.3.1
tensorflow version:  1.15.2


In [ ]:
#reading data

def load_data(fix_length, length):

  split_idx=55575
  #split_idx=135047
  data = pd.read_csv('/content/drive/MyDrive/Ali lotfy/Colab Notebooks/paper/Aqmar_fixed_features.csv')
  if(fix_length):
    for i, row in data.iterrows():
      data.at[i,'sentence_idx']=i//length
  for  i, row in data.iterrows():
    if data.at[i,"word"][-1] in ['*', ',', '.', ':', '=', '،', '؛' ] and len( data.at[i,"word"]) >1 : #remove punctuation if connected to word
      data.at[i, "word"]= data.at[i, "word"][0:-1]
    if 'ـ' in data.at[i,"word"] and len( data.at[i,"word"]) >2 : #remove tatweel تطويــــل
      data.at[i, "word"]= data.at[i, "word"] .replace("ـ", "")

  data_tr=data[0:split_idx]
  data_te=data[split_idx:]
  
  return data, data_tr, data_te

In [ ]:

# every sentence in a an array
class SentenceGetter(object):
    
    def __init__(self, dataset):
        self.n_sent = 1
        self.dataset = dataset
        self.empty = False
        agg_func = lambda s: [(w, t) for w,t in zip(s["word"].values.tolist(),
                                                        s["tag"].values.tolist())]
        self.grouped = self.dataset.groupby("sentence_idx").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None
def find_last(list , word, sent_len):
  l= sent_len-1
  while True:
    if list[l][0]==word:
      return l
    if l < int(sent_len*3/5):
      return sent_len-1
    else:
      l-=1

def group_sentences(sentences1, sent_len):
  sentences_tr=[]
  sent=[]
  i=0
  while (i<len(sentences1)):
    sent= sentences1[i]
    if len(sent) > sent_len:
      split= find_last(sent, "،", sent_len) +1
      sentences_tr.append(sent[0:split])
      sent=sent[split:]
      if (len(sent)<sent_len):
        try:
          while (len(sent)+len(sentences1[i+1])<=sent_len):
            sent=sent+sentences1[i+1]
            i+=1
          sentences_tr.append(sent)
        except:
          sentences_tr.append(sent)
      else:
        countVar=0
        while(countVar*sent_len<len(sent)):
          sentTemp=sent[sent_len*countVar:sent_len*(countVar+1)]
          sentences_tr.append(sentTemp)
          countVar+=1

    else:
      try:
        while (len(sent)+len(sentences1[i+1])<=sent_len):
          sent=sent+sentences1[i+1]
          i+=1
        sentences_tr.append(sent)
      except:
          sentences_tr.append(sent)

    i+=1
  return sentences_tr


def count2d(sentences):
  count=0
  for x in sentences:
    count+=len(x)
  return count


def load_sentences( sent_len):

  getter1 = SentenceGetter(data_tr)
  sentences1 = getter1.sentences
  getter2 = SentenceGetter(data_te)
  sentences2 = getter2.sentences

  sentences_tr = group_sentences(sentences1, sent_len)
  sentences_te = group_sentences(sentences2, sent_len)


  ### paddding sentences to have equal length
  sentences_tr= pad_sequences(maxlen=sent_len, sequences=sentences_tr, padding="post",value=('ENDPAD', 'O'),dtype=object)
  sentences_te= pad_sequences(maxlen=sent_len, sequences=sentences_te, padding="post",value=('ENDPAD', 'O'),dtype=object)
  sentences   = np.concatenate((sentences_tr , sentences_te), axis =0)
  ratio = np.shape(sentences_tr)[0] /(np.shape(sentences_te)[0]+np.shape(sentences_tr)[0]) 


  print("shape of training sentences: ",np.shape(sentences_tr))
  print("total padding: " , (np.shape(sentences_tr)[0]*np.shape(sentences_tr)[1])- count2d(sentences1))
  print("shape of testing sentences: ",np.shape(sentences_te))
  print("total padding: " , (np.shape(sentences_te)[0]*np.shape(sentences_te)[1])- count2d(sentences2))
  print("shape of all sentences: ",np.shape(sentences))

  return sentences, ratio

In [ ]:
#creating word and tag dictionary 
  
def create_dictionaries():
  words = list((set((data["word"]).values)))
  words.append("ENDPAD")
  n_words = len(words); 
  print("number of unique words is: ",n_words)

  '''tags = []
  for tag in set(data["tag"].values):
      if tag is nan or isinstance(tag, float):
          tags.append('unk')
      else:
          tags.append(tag)'''
  tags=['I-LOC', 'I-MISC', 'B-MISC','B-ORG', 'B-PERS', 'O', 'B-LOC', 'I-ORG', 'I-PERS']

  print("tags that we have: \n",tags)
  n_tags = len(tags); 
  print( 'number of tags is: ', n_tags)

  from future.utils import iteritems
  word2idx = {w: i for i, w in enumerate(words)}
  tag2idx = {t: i for i, t in enumerate(tags)}
  idx2tag = {v: k for k, v in iteritems(tag2idx)} 
  #print("index of word إبراهيم :",word2idx['علي'])
  print("index of tag I_PERS: ",tag2idx["I-PERS"])

  ##replace words and tags with indecies
  X_words = [[w[0] for w in s] for s in sentences]
  X_idx = [[word2idx[w[0]] for w in s] for s in sentences]  ##

  Y_idx = [[tag2idx[w[1]] for w in s] for s in sentences]
  Y_oneHot = [to_categorical(i, num_classes=n_tags) for i in Y_idx]

  print("shape of X_words", np.shape(X_words) )
  print("shape of X_idx", np.shape(X_idx) )
  print("shape of Y_idx", np.shape(Y_idx) )
  print("shape of Y_oneHot", np.shape(Y_oneHot) )

  return X_words, X_idx, Y_idx, Y_oneHot, words, tags, word2idx, tag2idx, idx2tag

In [ ]:
##======================methods for generating embeddings ==========================================
# parameters 
#
''' How to call theses methods
def arabert(X_words, model, layers, layer_mean, subtoken_pooling, fine_tune, use_context, size):
arabert(X_words, "aubmindlab/bert-large-arabertv02", "all", True,  "mean", False, False,1024 )
fasttext(X_words)
aravec(X_words)
glove(X_words)'''


#===================================================================================================
def fasttext(X_words):
  stacked_embedding = WordEmbeddings('ar')
  embedding_name="FastText"

  #adding arabert embeddings
  arabert_sents=[]
  for sent in X_words:
    text=""
    for w in sent:
      if( w != "ENDPAD"):
        text+=w
        text+=" "
    if(len(text.rstrip().split(" ")) >sent_len):
      print(len(text.rstrip().split(" ")))  
    bert_sent=Sentence(text=text.rstrip(), use_tokenizer=False)
    arabert_sents.append(bert_sent)


  for i, sent in enumerate(arabert_sents):
    print(i)
    stacked_embedding.embed(sent)

      
  X_arabert=[]
  for sent in arabert_sents:
    sent_vect=[]
    for w in sent:
      we=w.embedding.cpu().detach().numpy()
      if(len(we)!=300):
        print(w)
      sent_vect.append(we)
    X_arabert.append(sent_vect)


  print(np.shape(X_arabert))

  tt =X_arabert[0][0]
  tt[0:]=0
  X_arabert= pad_sequences(maxlen=sent_len, sequences=X_arabert, padding="post",value=tt,dtype=object)
  print(np.shape(X_arabert))

  return X_arabert, embedding_name
#===================================================================================================
def glove(X_words):
  #f = open("/content/drive/MyDrive/Colab Notebooks/glove_ANER2.txt", "r")
  f = open("/content/drive/MyDrive/Ali lotfy/ANER_embeddings/glove_ANER2.txt", "r")

  aravec_embeddings=[]
  embedding_name="GloVE"
  for line in f:
    stringArray = np.array(line.split(" ")[1:])
    floatArray = stringArray.astype(float)
    aravec_embeddings.append(floatArray)

  #f = open("/content/drive/MyDrive/Colab Notebooks/glove_ANER2.txt", "r")
  f = open("/content/drive/MyDrive/Ali lotfy/ANER_embeddings/glove_ANER2.txt", "r")
  
  myWords=[]
  for line in f:
    myWords.append(line.split(" ")[0])

  X_aravec=[]
  count=0
  for sentence in X_words:
    count+=1
    print(count)
    sen=[]
    for w in sentence: 
        if w in myWords:
          try:
            sen.append(aravec_embeddings[myWords.index(w)])
          except:
            sen.append(np.zeros(256))
        else:
            sen.append(np.zeros(256))    
    X_aravec.append(sen)
    
  print(np.shape(X_aravec))
  return X_aravec, embedding_name
#===================================================================================================
def aravec(X_words):
  #f = open("/content/drive/MyDrive/Colab Notebooks/aravec_300.txt", "r")
  f = open("/content/drive/MyDrive/Ali lotfy/ANER_embeddings/aravec_300.txt", "r")
  aravec_embeddings=[]
  embedding_name="AraVec"

  for line in f:
    stringArray = np.array(line.split(" ")[1:])
    floatArray = stringArray.astype(float)
    aravec_embeddings.append(floatArray)

 #f = open("/content/drive/MyDrive/Colab Notebooks/aravec_300.txt", "r")
  f = open("/content/drive/MyDrive/Ali lotfy/ANER_embeddings/aravec_300.txt", "r")
  
  myWords=[]
  for line in f:
    myWords.append(line.split(" ")[0])

  X_aravec=[]
  count=0
  for sentence in X_words:
    count+=1
    print(count)
    sen=[]
    for w in sentence: 
        if w in myWords:
          try:
            sen.append(aravec_embeddings[myWords.index(w)])
          except:
            sen.append(np.zeros(300))
        else:
            sen.append(np.zeros(300))    
    X_aravec.append(sen)
    
  print(np.shape(X_aravec))
  return X_aravec, embedding_name

#===================================================================================================
def arabert(X_words, model, layers, layer_mean, subtoken_pooling, fine_tune, use_context, size):
  stacked_embedding = StackedEmbeddings([   
    TransformerWordEmbeddings(model=model, layers=layers, layer_mean=layer_mean,
                              subtoken_pooling=subtoken_pooling, fine_tune = fine_tune, use_context =use_context )
    ])
  if (size== 1024):
    embedding_name="AraBertLarge"
  elif (size==768):
    embedding_name="AraBertBase"
  

  #adding arabert embeddings
  arabert_sents=[]
  for sent in X_words:
    text=""
    for w in sent:
      if( w != "ENDPAD"):
        text+=w
        text+=" "
    if(len(text.rstrip().split(" ")) >sent_len):
      print(len(text.rstrip().split(" ")))  
    bert_sent=Sentence(text=text.rstrip(), use_tokenizer=False)
    arabert_sents.append(bert_sent)


  for i, sent in enumerate(arabert_sents):
    print(i)
    stacked_embedding.embed(sent)

      
  X_arabert=[]
  for sent in arabert_sents:
    sent_vect=[]
    for w in sent:
      we=w.embedding.cpu().detach().numpy()
      if(len(we)!=size):
        print(w)
      sent_vect.append(we)
    X_arabert.append(sent_vect)


  print(np.shape(X_arabert))

  tt =X_arabert[0][0]
  tt[0:]=0
  X_arabert= pad_sequences(maxlen=sent_len, sequences=X_arabert, padding="post",value=tt,dtype=object)
  print(np.shape(X_arabert))


  return X_arabert, embedding_name

  #========================== Flair embeddings =======================================

def flair(X_words, model):
  stacked_embedding = StackedEmbeddings([   
      FlairEmbeddings(model)
    ])
  embedding_name=model

  #adding arabert embeddings
  arabert_sents=[]
  for sent in X_words:
    text=""
    for w in sent:
      if( w != "ENDPAD"):
        text+=w
        text+=" "
    if(len(text.rstrip().split(" ")) >sent_len):
      print(len(text.rstrip().split(" ")))  
    bert_sent=Sentence(text=text.rstrip(), use_tokenizer=False)
    arabert_sents.append(bert_sent)


  for i, sent in enumerate(arabert_sents):
    print(i)
    stacked_embedding.embed(sent)

      
  X_arabert=[]
  for sent in arabert_sents:
    sent_vect=[]
    for w in sent:
      we=w.embedding.cpu().detach().numpy()
      sent_vect.append(we)
    X_arabert.append(sent_vect)


  print(np.shape(X_arabert))

  tt =X_arabert[0][0]
  tt[0:]=0
  X_arabert= pad_sequences(maxlen=sent_len, sequences=X_arabert, padding="post",value=tt,dtype=object)
  print(np.shape(X_arabert))


  return X_arabert, embedding_name





In [ ]:
##================================== =========================== Character embedding =========================== =====================================
#character embedding enum
char_len=16
from enum import Enum
class Char_type(Enum):
  NONE = "NONE"
  LSTM = "LSTM"
  CNN = "CNN"

'''from IPython.display import clear_output

chars = set([w_i for w in words for w_i in w])

char_dict={}
for c in chars:
  print(c)
  x=input()
  char_dict[c]=x
  clear_output()
  '''

  

def char(X_words):

  char_dict ={'!': '.', '"': '.', '#': '.', '%': '.', '&': '.', "'": '.', '(': '(', ')': ')', '*': '.', '+': '.', ',': '.', '-': '.',
              '.': '.', '/': '.', '0': '0', '1': '0', '2': '0', '3': '0', '4': '0', '5': '0', '6': '0', '7': '0', '8': '0', '9': '0',
              ':': '.', '=': '.', '>': '.', '?': '.', 'A': 'x', 'B': 'x', 'C': 'x', 'D': 'x', 'E': 'x', 'F': 'x', 'G': 'x', 'H': 'x',
              'I': 'x', 'J': 'x', 'K': 'x', 'L': 'x', 'M': 'x', 'N': 'x', 'O': 'x', 'P': 'x', 'R': 'x', 'S': 'x', 'T': 'x', 'U': 'x',
              'V': 'x', 'W': 'x', 'X': 'x', 'Y': 'x', 'Z': 'x', '[': '(', ']': ')', 'a': 'x', 'b': 'x', 'c': 'x', 'd': 'x', 'e': 'x',
              'f': 'x', 'g': 'x', 'h': 'x', 'i': 'x', 'k': 'x', 'l': 'x', 'm': 'x', 'n': 'x', 'o': 'x', 'p': 'x', 'q': 'x', 'r': 'x',
              's': 'x', 't': 'x', 'u': 'x', 'v': 'x', 'w': 'x', 'x': 'x', 'y': 'x', '«': '.', '»': '.', 'í': 'x', 'ü': 'x', '،': '.',
              '؛': '.', '؟': '.', 'ء': 'ء', 'آ': 'ء ا', 'أ': 'ء', 'ؤ': 'ء', 'إ': 'ء', 'ئ': 'ء', 'ا': 'ا', 'ب': 'ب', 'ة': 'ة', 'ت': 'ت',
              'ث': 'ث', 'ج': 'ج', 'ح': 'ح', 'خ': 'خ', 'د': 'د', 'ذ': 'ذ', 'ر': 'ر', 'ز': 'ز', 'س': 'س', 'ش': 'ش', 'ص': 'ص', 'ض': 'ض',
              'ط': 'ط', 'ظ': 'ظ', 'ع': 'ع', 'غ': 'غ', 'ـ': '.', 'ف': 'ف', 'ق': 'ق', 'ك': 'ك', 'ل': 'ل', 'م': 'م', 'ن': 'ن', 'ه': 'ه',
              'و': 'و', 'ى': 'ا', 'ي': 'ي', '–': '.', '’': '.', '“': '.', '”': '.', '…': '.', 'ﻷ': 'ل ء', 'ﻹ': 'ل ء'}

  chars = set([w_i for w in words for w_i in w])
  new_chars=set([char_dict[w] for w in chars ])
  n_chars = len(new_chars)
  print("number of characters is :", n_chars)


  char2idx = {c: i + 2 for i, c in enumerate(new_chars)}
  char2idx["UNK"] = 1
  char2idx["PAD"] = 0


  X_char = []
  for sentence in X_words:
    sent_seq = []
    for i in range(len(sentence)): #sequence length
        word_seq = []
        j=0
        var=0
        while j < char_len - var:
          try:
            char=char_dict[sentence[i][j]]
            if (char2idx.get(char)==None):
              word_seq.append(0) ## char2idx["PAD"] = 0
            else:
              char=char_dict[sentence[i][j]]
              if (len(char)==3):
                char1=char_dict[char[0]]
                char1= char2idx.get(char1)
                word_seq.append(char1)

                char2=char_dict[char[2]]
                char2= char2idx.get(char2)
                word_seq.append(char2)
                var+=1

              else:
                char= char2idx.get(char)
                word_seq.append(char)
          except:
            word_seq.append(0)

          j+=1
        sent_seq.append(word_seq)
    X_char.append(np.array(sent_seq))

  print(np.shape(X_char))

  return X_char, n_chars



def char1(X_words):
  chars = set([w_i for w in words for w_i in w])
  n_chars = len(chars)
  print("number of characters is :", n_chars)

  char2idx = {c: i + 2 for i, c in enumerate(chars)}
  char2idx["UNK"] = 1
  char2idx["PAD"] = 0
  X_char = []
  for sentence in sentences:
      sent_seq = []
      for i in range(sent_len): #sequence length
          word_seq = []
          for j in range(char_len):
              try:
                  if(char2idx.get(sentence[i][0][j])==None):
                    word_seq.append(0) ## char2idx["PAD"] = 0
                  else:
                    word_seq.append(char2idx.get(sentence[i][0][j]))
              except:
                  word_seq.append(0)
          sent_seq.append(word_seq)
      X_char.append(np.array(sent_seq))

  return X_char, n_chars

In [ ]:

##================================== =========================== ALL Features =========================== =====================================


max_fea_len=123
quote1 =['«', '"', '(']
quote2 =['»', '"', ')']







def features1( X_words):
  X_fea = []
  i=0
  idx=0
  add_quote = False
  quote_count = 0
  quote_index=-1
  even_flag=True, True, True ##to assure the second quote is not considered
  while i<len(X_words):
    sent_seq = []
    j=0
    sent=X_words[i]
    while (j<len(sent)):
      word=sent[j]
      if (word=="ENDPAD"):
        feature_vector = np.zeros(max_fea_len)
      else:
        feature_vector=[]
        feature_vector+= one_hot1(data.at[idx,'pos'], pos_values)
        feature_vector+= one_hot1(data.at[idx,'enc0'], enc0_values)
        feature_vector+= one_hot1(data.at[idx,'stt'], stt_values)
        feature_vector+= one_hot1(data.at[idx,'num'], num_values)
        feature_vector+= one_hot1(data.at[idx,'gen'], gen_values)
        feature_vector+= one_hot1(data.at[idx,'prc0'], prc0_values)
        feature_vector+= one_hot1(data.at[idx,'prc1'], prc1_values)
        feature_vector+= one_hot1(data.at[idx,'prc2'], prc2_values)
        feature_vector+= one_hot1(data.at[idx,'prc3'], prc3_values)
        feature_vector+= one_hot1(data.at[idx,'per'], per_values)
        feature_vector+= one_hot1(data.at[idx,'vox'], vox_values)
        feature_vector+= one_hot1(data.at[idx,'asp'], asp_values)
        feature_vector+= one_hot1(data.at[idx, 'cas'], cas_values)
        feature_vector+= one_hot1(data.at[idx, 'mod'], mod_values)

        try:
          if(data.at[idx,"gloss"][0].isupper()):
            feature_vector+=[1.0]
          else:
            feature_vector+=[0.0]
        except:
          feature_vector+=[0.0]
        idx+=1
        ##quote feature
        if add_quote :   ## add values
          feature_vector+= [quote_index+1]
          feature_vector+= [quote_count]
          quote_count -= 1
        else:
          feature_vector+=[0,0]

        if not add_quote and word in quote1: # check if words are between quotes
          quote_index= quote1.index(word)
          if (even_flag or word!='"'):
            for k in [1,2,3,4]:
              try:
                if sent[i+k] == quote2[quote_index]:
                  add_quote= True
                  quote_count=4
                  break
              except:
                add_quote= False
          if (word =='"'):  #to avoid the second quote
            even_flag= not even_flag
             
        if (quote_count ==1): #stop adding values
          add_quote = False
      sent_seq.append(feature_vector)
      j+=1
      
    X_fea.append(sent_seq)
    i+=1
  
  print(np.shape(X_fea))
  return X_fea



def one_hot1( value, list):
  x=np.zeros(len(list))
  idx= list.index(value)
  x[idx]=1
  return x.tolist()

pos_values= ['pron_interrog', 'part_fut', 'punc', 'part', 'noun', 'adv', 'interj', 'verb', 'conj_sub', 'adv_interrog', 'noun_prop', 'verb_pseudo', 'part_focus', 'part_interrog', 'conj', 'adj', 'part_voc', 'digit', 'abbrev', 'pron', 'pron_rel', 'part_det', 'part_neg', 'part_verb', 'prep', 'pron_dem', 'adv_rel', 'pron_exclam', 'noun_quant']
cas_values= ['na', 'n', 'u', 'a', 'g']
enc0_values= ['3fp_poss', '3fp_pron', '2mp_poss', 'mA_sub', '3mp_pron', '3ms_dobj', '1s_dobj', '2ms_poss', '1s_pron', '3d_dobj', '1p_dobj', '2ms_pron', '3mp_dobj', '1p_poss', 'ma_interrog', '2fs_pron', '2mp_dobj', '2ms_dobj', 'man_rel', '3ms_poss', '3mp_poss', '3d_poss', '3fs_dobj', '3fs_poss', '2fp_poss', '1p_pron', 'mA_rel', 'lA_neg', '3d_pron', 'na', '2fs_poss', '3ms_pron', '1s_poss', '2fs_dobj', '0', '3fp_dobj', '3fs_pron', '2mp_pron']
stt_values= ['c', 'i', 'd', 'na']
num_values= ['d', 'p', 'na', 's']
gen_values= ['f', 'm', 'na']
vox_values= ['a', 'p', 'na']
asp_values= ['c', 'i', 'p', 'na']
per_values= ['1', '3', 'na', '2']
prc0_values= ['mA_rel', 'mA_neg', 'lA_neg', 'na', 'Al_det', '0']
prc1_values= ['na', 'ka_prep', 'bi_prep', 'li_prep', 'la_emph', '0', 'la_rc', 'sa_fut']
prc2_values = ['na', 'wa_sub', 'wa_conj', 'wa_part', 'fa_conj', '0']
prc3_values= ['>a_ques', '0', 'na']
mod_values= ['i', 'na', 'u']


In [ ]:
#create model
def create_model(lstm_units, embedding_dropout, lstm_dropout,activation_function, optimizer, 
                 dense_dropout , second_layer, dense_units, variational_dropout,):

  ##word input 
  if (emb_size==1):
    word_input = Input(shape=(sent_len,),name='word_input') ##sent_len is sentence length 
    embeddings=Embedding(input_dim=len(words) + 1, output_dim=20,)(word_input)
  else:
    word_input = Input(shape=(sent_len,emb_size,),name='word_input') ##sent_len is sentence length 
    embeddings=word_input

  
  ##Feature input 
  if (use_features):
    feature_input = Input(shape=(sent_len,max_fea_len,),name='feature_input')
    embeddings = concatenate([embeddings, feature_input])

  ## Character embedding
  if (character_embedding  is Char_type.LSTM):
    char_input = Input(shape=(sent_len, char_len,),name='char_input')
    emb_char = TimeDistributed(Embedding(input_dim=n_chars + 2 , output_dim=25, 
                              input_length=char_len))(char_input)
    char_enc = TimeDistributed(LSTM(units=25, return_sequences=False, 
                                    recurrent_dropout=0.5))(emb_char)
    embeddings = concatenate([embeddings, char_enc])

  if (character_embedding  is Char_type.CNN):
    char_input=Input(shape=(sent_len,char_len,1,),name='char_input')
    emb_char=TimeDistributed(Embedding(input_dim=n_chars + 2,output_dim=25,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(char_input)
    conv2d_out= TimeDistributed(Conv2D(kernel_size=3, filters=25, padding='same',activation='tanh', strides=1))(emb_char)
    maxpool_out=TimeDistributed(MaxPooling2D(1))(conv2d_out)
    char_enc = TimeDistributed(Flatten())(maxpool_out)
    embeddings = concatenate([embeddings, char_enc])

  ##dropout layer on embedding layer
  model= Dropout(embedding_dropout, trainable = variational_dropout)(embeddings)

  
  ## 2 bidirectional LSTM layers 
  model = Bidirectional(LSTM(units=lstm_units, 
                            return_sequences=True, 
                            dropout=lstm_dropout, 
                            recurrent_dropout=lstm_dropout, 
                            kernel_initializer=keras.initializers.he_normal()))(model)
  if (second_layer):
    model = (LSTM(units=lstm_units*2 , 
                            return_sequences=True, 
                            dropout=lstm_dropout,
                            recurrent_dropout=lstm_dropout, 
                            kernel_initializer=keras.initializers.he_normal()))(model)


  model = TimeDistributed(Dense(dense_units, activation=activation_function))(model)  # previously softmax output layer
  model= Dropout(dense_dropout)(model)
  '''if (use_features):
    feature_input = Input(shape=(sent_len,max_fea_len,),name='feature_input')
    model = concatenate([model, feature_input])'''
  ##CRF layer
  crf = CRF(9)  
  out = crf(model) 

  
  if (use_features and character_embedding  in [Char_type.LSTM, Char_type.CNN]):
      inputs = [word_input,char_input, feature_input]
  elif (character_embedding  in [Char_type.LSTM, Char_type.CNN]):
      inputs = [word_input,char_input]
  elif (use_features):
      inputs = [word_input,feature_input]
  else:
    inputs = word_input

  ##compile model 
  model = Model(inputs, out)

  model.compile(optimizer=optimizer, loss=crf.loss_function, metrics=[crf.accuracy, "accuracy"])

  model.summary()


 
  return model


def train_model(model, batch, epochs,validation_split, callbacks_list):
  if (use_features and character_embedding  is Char_type.LSTM ):
    history = model.fit([np.array(X_train), np.array(X_char_tr), np.array(X_fea_tr)],
                        np.array(Y_train),
                        batch_size=batch,
                        epochs=epochs,
                        validation_split=validation_split,
                        verbose=1,
                        callbacks=callbacks_list)
  elif (character_embedding is Char_type.LSTM):
    history = model.fit([np.array(X_train), np.array(X_char_tr)],
                        np.array(Y_train),
                        batch_size=batch,
                        epochs=epochs,
                        validation_split=validation_split,
                        verbose=1,
                        callbacks=callbacks_list)
  elif (use_features and character_embedding  is Char_type.CNN ):
    history = model.fit([np.array(X_train), np.array(X_char_tr).reshape(len(X_char_tr), sent_len, char_len, 1), np.array(X_fea_tr)],
                        np.array(Y_train),
                        batch_size=batch,
                        epochs=epochs,
                        validation_split=validation_split,
                        verbose=1,
                        callbacks=callbacks_list)
  elif (character_embedding is Char_type.CNN):
    history = model.fit([np.array(X_train), np.array(X_char_tr).reshape(len(X_char_tr), sent_len, char_len, 1)],
    #history = model.fit( np.array(X_char_tr).reshape(len(X_char_tr), sent_len, char_len, 1),

                        np.array(Y_train),
                        batch_size=batch,
                        epochs=epochs,
                        validation_split=validation_split,
                        verbose=1,
                        callbacks=callbacks_list)
    
  elif (use_features):
    history = model.fit([np.array(X_train), np.array(X_fea_tr)],
                        np.array(Y_train),
                        batch_size=batch,
                        epochs=epochs,
                        validation_split=validation_split,
                        verbose=1,
                        callbacks=callbacks_list)
  else:

    history = model.fit(np.array(X_train),
                        np.array(Y_train),
                        batch_size=batch,
                        epochs=epochs,
                        validation_split=validation_split,
                        verbose=1,
                        callbacks=callbacks_list)
    
def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i])
        out.append(out_i)
    return out

def eval_model(model):
  if (use_features and character_embedding  is Char_type.LSTM):
    test_pred = model.predict([np.array(X_test), np.array(X_char_te), np.array(X_fea_te) ], verbose=1)   

  elif (character_embedding  is Char_type.LSTM):
    test_pred = model.predict([np.array(X_test), np.array(X_char_te) ], verbose=1)

  elif (use_features and character_embedding  is Char_type.CNN):
    test_pred = model.predict([np.array(X_test), np.array(X_char_te).reshape(len(X_char_te), sent_len, char_len, 1), np.array(X_fea_te) ], verbose=1)   

  elif (character_embedding  is  Char_type.CNN):
    test_pred = model.predict([np.array(X_test), np.array(X_char_te).reshape(len(X_char_te), sent_len, char_len, 1) ], verbose=1)   

  elif (use_features):
    test_pred = model.predict([np.array(X_test), np.array(X_fea_te) ], verbose=1)   

  else:
    test_pred = model.predict(np.array(X_test), verbose=1)   
  pred_labels = pred2label(test_pred)
  test_labels = pred2label(Y_test)
  results=[]
  results+=[round(precision_score(test_labels, pred_labels)*100, 2)]
  results+=[round(recall_score(test_labels, pred_labels)*100, 2)]
  results+=[round(f1_score(test_labels, pred_labels)*100, 2)]
  return(results)
  #classification_report(test_labels, pred_labels)

In [ ]:
## method to split data

def split(X_embeddings):
  try:
    emb_size = np.shape(X_embeddings)[2]
  except:
    emb_size =1
  results=[]

  if(random_split):  ## random split 
    if (use_features and character_embedding  in [Char_type.LSTM, Char_type.CNN]):
      X_fin = np.dstack((X_embeddings,X_char, X_fea))

    elif (use_features):
      X_fin = np.dstack((X_embeddings, X_fea))

    elif (character_embedding  in [Char_type.LSTM, Char_type.CNN]):
      X_fin = np.dstack((X_embeddings,X_char))

    else:
      X_fin=X_embeddings


    X_train, X_test, Y_train, Y_test = train_test_split(X_fin, Y_oneHot, test_size=0.1,random_state=2018)

    if (use_features and character_embedding  in [Char_type.LSTM, Char_type.CNN]):
      X_fea_tr =[[w[emb_size + char_len:] for w in s] for s in X_train]
      X_fea_te =[[w[emb_size + char_len:] for w in s] for s in X_test]
      X_char_tr =[[w[emb_size:emb_size + char_len] for w in s] for s in X_train]
      X_char_te =[[w[emb_size:emb_size + char_len] for w in s] for s in X_test]
      X_train =[[w[0:emb_size] for w in s] for s in X_train]
      X_test =[[w[0:emb_size] for w in s] for s in X_test]
      results +=[X_char_tr, X_char_te, X_fea_tr, X_fea_te]

      print("X_char_tr ",np.shape(X_char_tr))
      print("X_char_te ",np.shape(X_char_te))
      print("X_fea_tr ",np.shape(X_fea_tr))
      print("X_fea_te ", np.shape(X_fea_te))
    elif (use_features):
      X_fea_tr =[[w[emb_size :] for w in s] for s in X_train]
      X_fea_te =[[w[emb_size:] for w in s] for s in X_test]
      X_train =[[w[0:emb_size] for w in s] for s in X_train]
      X_test =[[w[0:emb_size] for w in s] for s in X_test]
      results +=[X_fea_tr, X_fea_te]

      print("X_fea_tr ",np.shape(X_fea_tr))
      print("X_fea_te ", np.shape(X_fea_te))
    elif (character_embedding  in [Char_type.LSTM, Char_type.CNN]):
      X_char_tr =[[w[emb_size:] for w in s] for s in X_train]
      X_char_te =[[w[emb_size:] for w in s] for s in X_test]
      X_train =[[w[0:emb_size] for w in s] for s in X_train]
      X_test =[[w[0:emb_size] for w in s] for s in X_test]
      results +=[X_char_tr, X_char_te]

      print("X_char_tr ",np.shape(X_char_tr))
      print("X_char_te ",np.shape(X_char_te))
    ## these to lines if one hot embeddings are used 
    #X_train= [[f[0] for f in e] for e in X_train ]
    #X_test= [[f[0] for f in e] for e in X_test ]


    results =[X_train, X_test, Y_train, Y_test]+results +[emb_size]

    print("X_train ",np.shape(X_train))
    print("X_test ",np.shape(X_test))
    print("Y_train ", np.shape(X_test))
    print("Y_test ", np.shape(Y_test))

    return results


  else: # camel split
    print("X_embedding ",np.shape(X_embeddings)) 
    print("Y_oneHot ", np.shape(Y_oneHot))

    split_idx =int(ratio*np.shape(sentences)[0])


    X_train    =X_embeddings   [0:split_idx]
    X_test     =X_embeddings    [split_idx: ]
    Y_train    =Y_oneHot [0:split_idx]
    Y_test     =Y_oneHot [split_idx: ]
    print("X_train ",np.shape(X_train))
    print("X_test ",np.shape(X_test))
    print("Y_train ", np.shape(Y_train))
    print("Y_test ", np.shape(Y_test))
    results+=[X_train, X_test, Y_train, Y_test]


    if ( character_embedding  in [Char_type.LSTM, Char_type.CNN]):
      X_char_tr    =X_char    [0:split_idx]
      X_char_te  =X_char    [split_idx: ]
      
      print("X_char_tr ",np.shape(X_char_tr))
      print("X_char_tr ",np.shape(X_char_te))
      results+=[X_char_tr, X_char_te]

    if (use_features):
      X_fea_tr    =X_fea    [0:split_idx]
      X_fea_te  =X_fea    [split_idx: ]
      
      print("X_fea_tr ",np.shape(X_fea_tr))
      print("X_fea_te ", np.shape(X_fea_te))
      results+=[X_fea_tr, X_fea_te]

    return results+ [emb_size]






In [ ]:
##method that runs the model

def experiment(count, repeate, name, epochs):
  
  model= create_model(lstm_units, embedding_dropout, lstm_dropout,activation_function, optimizer,dense_dropout , second_layer, dense_units, variational_dropout)
  epochs_temp=epochs
  epochs=0
  highest=0
  NoImprove=0
  df = pd.DataFrame(columns = ['embedding', 'dimension', 'features', 'character','F1', 'Precision', 'Recall',
                             'epochs', 'batch','validation', 'activation function', 'optimizer','lstm units',    
                             'sentence length', 'fixed length', 'emb dropout', 'lstm dropout', 'dense dropout', 'second layer',
                             'dense units','variational_dropout' 
                             ])
  results=""
  path="/content/drive/MyDrive/Ali lotfy/saved models AQMAR/"+name
  path_temp="/content/drive/MyDrive/Ali lotfy/saved models AQMAR/"+name
  os.mkdir(path)
  for o in range(count):
    if (repeate):
      model= create_model(lstm_units, embedding_dropout, lstm_dropout,activation_function, optimizer,dense_dropout , second_layer, dense_units, variational_dropout)
      epochs= epochs_temp
    else:
      epochs+= epochs_temp


    if (NoImprove<9):
      es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
      reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=2, min_lr=0.0001)
      callbacks_list=[ es, reduce_lr]
      train_model(model=model, batch=batch_size, epochs=epochs_temp, validation_split=val, callbacks_list=callbacks_list )
      
      eval=eval_model(model)
      

      if (eval[2]> highest or o%10==9 ):
        df = df.append({'embedding': name, 'dimension': emb_size, 'features': use_features,
                    'character': character_embedding.name, 'F1': eval[2], 'Precision': eval[0], 'Recall': eval[1],
                    'epochs': epochs, 'batch': batch_size, 'validation': val, 'activation function': activation_function,
                    'optimizer': optimizer, 'lstm units': lstm_units, 'sentence length': sent_len,'fixed length': fix_length, 'emb dropout': embedding_dropout,
                    'lstm dropout': lstm_dropout, 'dense dropout': dense_dropout, 'second layer': second_layer,
                    'dense units': dense_units, 'variational_dropout': variational_dropout}, 
                    ignore_index = True)
        results+="F1-score= {} Precision= {} Recall={}".format(eval[2], eval[0], eval[1])
        results+=" epochs= {} lstm_units= {} embedding_dropout= {} lstm_dropout= {} activation_function= {} optimizer= {} dense_dropout= {} second_layer= {} dense_units= {} validation= {} batch_size= {} variational_dropout= {}".format(epochs,lstm_units,embedding_dropout,lstm_dropout,activation_function,optimizer,dense_dropout,second_layer,dense_units,val,batch_size, variational_dropout)
        results+="\n"
        print(results)
      
      if (eval[2]> highest):
        highest= eval[2]
        os.rename(path, path_temp+" "+str(highest))
        path=path_temp+" "+str(highest)
        save_model(model, weights_file=path+"/weights.h5", params_file=path+"/params.json")
        best={'embedding': name, 'dimension': emb_size, 'features': use_features,
                    'character': character_embedding.name, 'F1': eval[2], 'Precision': eval[0], 'Recall': eval[1],
                    'epochs': epochs, 'batch': batch_size, 'validation': val, 'activation function': activation_function,
                    'optimizer': optimizer, 'lstm units': lstm_units, 'sentence length': sent_len,'fixed length': fix_length, 'emb dropout': embedding_dropout,
                    'lstm dropout': lstm_dropout, 'dense dropout': dense_dropout, 'second layer': second_layer,
                    'dense units': dense_units, 'variational_dropout': variational_dropout}
        NoImprove=0
      else:
        NoImprove+=1


  dfAll = pd.read_csv('/content/drive/MyDrive/Ali lotfy/results_AQMAR.csv')
  dfAll=dfAll.append(best,ignore_index = True)
  try:
    dfAll=dfAll.drop('Unnamed: 0', axis=1)
  except:
    idle=0
  dfAll.to_csv("/content/drive/MyDrive/Ali lotfy/results_AQMAR.csv")  ## name of the experiment


  df.to_csv(path+"/results.csv")  ## name of the experiment
  return df
  
def save_model(model, weights_file, params_file):
    with open(params_file, 'w') as f:
        params = model.to_json()
        json.dump(json.loads(params), f, sort_keys=True, indent=4)
        model.save_weights(weights_file)


def load_model(weights_file, params_file):
    with open(params_file) as f:
        model = model_from_json(f.read(), custom_objects={'CRF': CRF})
        model.load_weights(weights_file)
    return model

In [ ]:
#=========load dataset, preproceesing=======================
fix_length=False
sent_len=100

data, data_tr, data_te= load_data(fix_length, sent_len)

sentences, ratio= load_sentences(sent_len)
X_words, X_idx, Y_idx, Y_oneHot, words, tags, word2idx, tag2idx, idx2tag=create_dictionaries()


#==========================embedding========================
#def arabert(X_words, model, layers, layer_mean, subtoken_pooling, fine_tune, use_context, size):
#X_arabert, X_arabert_name= arabert(X_words, "aubmindlab/bert-large-arabertv02", "all", True,  "mean", False, False,1024 )
X_fasttext, X_fasttext_name= fasttext(X_words)
#X_aravec, X_aravec_name = aravec(X_words)
#X_glove, X_glove_name= glove(X_words)
#X_flair, X_flair_name  =flair(X_words, "ar-forward")


#======================= characyer embedding and features =========
X_char, n_chars =char1(X_words)
#max_fea_len = 123      ## 123 for all, 1 for cap, 2 for quote, 29 for pos, 91 for analysis
X_fea=features1(X_words)


#============concatenate two embeddings ================
#X_embeddings= np.dstack((X_arabert, X_fasttext))
#embedding_name="ArabertLarge+ Fasttext"

shape of training sentences:  (681, 100, 2)
total padding:  12525
shape of testing sentences:  (228, 100, 2)
total padding:  4521
shape of all sentences:  (909, 100, 2)
number of unique words is:  17936
tags that we have: 
 ['I-LOC', 'I-MISC', 'B-MISC', 'B-ORG', 'B-PERS', 'O', 'B-LOC', 'I-ORG', 'I-PERS']
number of tags is:  9
index of tag I_PERS:  8
shape of X_words (909, 100)
shape of X_idx (909, 100)
shape of Y_idx (909, 100)
shape of Y_oneHot (909, 100, 9)
2022-04-28 07:59:37,659 https://flair.informatik.hu-berlin.de/resources/embeddings/token/ar-wiki-fasttext-300d-1M.vectors.npy not found in cache, downloading to /tmp/tmpu81wrs97


100%|██████████| 733171328/733171328 [01:23<00:00, 8775377.59B/s] 

2022-04-28 08:01:02,347 copying /tmp/tmpu81wrs97 to cache at /root/.flair/embeddings/ar-wiki-fasttext-300d-1M.vectors.npy


2022-04-28 08:01:04,779 removing temp file /tmp/tmpu81wrs97
2022-04-28 08:01:06,084 https://flair.informatik.hu-berlin.de/resources/embeddings/token/ar-wiki-fasttext-300d-1M not found in cache, downloading to /tmp/tmp7n_pop6n


100%|██████████| 26704903/26704903 [00:03<00:00, 7171734.64B/s]

2022-04-28 08:01:10,548 copying /tmp/tmp7n_pop6n to cache at /root/.flair/embeddings/ar-wiki-fasttext-300d-1M
2022-04-28 08:01:10,615 removing temp file /tmp/tmp7n_pop6n


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:1970: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


(909,)
(909, 100, 300)
number of characters is : 136
(909, 100, 123)


In [ ]:
`#fine tune arabertlargev02 with features_2019 and CNN, sent_len = 256 
#for lstm_units in [50, 100, 150, 200, 250, 300, 350, 400, 450, 500(), 550, 600, 650, 700, 750, 800, 750, 900]:
#for optimizer in ["adam", "nadam", "adagrad", "adamax", "RMSprop", "adadelta"]:
#for activation_function in ["relu", "tanh", "softplus", "linear", "softsign", "hard_sigmoid"]:
#for embedding_dropout in [0, 0.1, 0.2, 0.3, 0.4, 0.5]:
#for lstm_dropout in [0, 0.1, 0.2, 0.3, 0.4, 0.5]:
#for dense_dropout in [0, 0.1, 0.2, 0.3, 0.4, 0.5]:
#for dense_units in [10,32,64,100,150,200, 500]:
#for val in [0, 0.05, 0.11111111, 0.15, 0.2, 0.25 ]:
#for batch_size in [8, 16, 32, 64]:
#for sent_len in [50, 100 ,200, 300, 512]:


#sentence length fix length 
#epochs


In [ ]:

X_embeddings= X_fasttext 
embedding_name=X_fasttext_name

random_split=False
use_features=False 
character_embedding =Char_type("NONE")


if (use_features and character_embedding  in [Char_type.LSTM, Char_type.CNN]):
  X_train, X_test, Y_train, Y_test, X_char_tr, X_char_te, X_fea_tr, X_fea_te, emb_size=split(X_embeddings)

elif (use_features):
  X_train, X_test, Y_train, Y_test, X_fea_tr, X_fea_te, emb_size=split(X_embeddings)

elif (character_embedding  in [Char_type.LSTM, Char_type.CNN]):
  X_train, X_test, Y_train, Y_test, X_char_tr, X_char_te, emb_size=split(X_embeddings)

else:
  X_train, X_test, Y_train, Y_test, emb_size=split(X_embeddings)


lstm_units=200 #500
activation_function="hard_sigmoid" # relu
embedding_dropout=0.3
lstm_dropout=0.5
optimizer="nadam"
dense_dropout=0.3 
second_layer=False
dense_units=150
val=0.0
batch_size=5
epochs=1
variational_dropout= False

for cc in range(6):
    df1= experiment(40, False, "FastText", epochs) ## experiment ( number of epochs, repeate runs or increase epochs , name of csv file)


X_embedding  (909, 100, 300)
Y_oneHot  (909, 100, 9)
X_train  (681, 100, 300)
X_test  (228, 100, 300)
Y_train  (681, 100, 9)
Y_test  (228, 100, 9)


/usr/local/lib/python3.7/dist-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/usr/local/lib/python3.7/dist-packages/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


Model: "model_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
word_input (InputLayer)      (None, 100, 300)          0         
_________________________________________________________________
dropout_59 (Dropout)         (None, 100, 300)          0         
_________________________________________________________________
bidirectional_31 (Bidirectio (None, 100, 400)          801600    
_________________________________________________________________
time_distributed_65 (TimeDis (None, 100, 150)          60150     
_________________________________________________________________
dropout_60 (Dropout)         (None, 100, 150)          0         
_________________________________________________________________
crf_29 (CRF)                 (None, 100, 9)            1458      
Total params: 863,208
Trainable params: 863,208
Non-trainable params: 0
____________________________________________________

/tensorflow-1.15.2/python3.7/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,crf_viterbi_accuracy,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
/tensorflow-1.15.2/python3.7/keras/callbacks/callbacks.py:1042: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,crf_viterbi_accuracy,accuracy,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


228/228 [==============================] - 8s 36ms/step
F1-score= 19.08 Precision= 30.58 Recall=13.87 epochs= 1 lstm_units= 200 embedding_dropout= 0.3 lstm_dropout= 0.5 activation_function= hard_sigmoid optimizer= nadam dense_dropout= 0.3 second_layer= False dense_units= 150 validation= 0.0 batch_size= 5 variational_dropout= False

Epoch 1/1
228/228 [==============================] - 1s 5ms/step
F1-score= 19.08 Precision= 30.58 Recall=13.87 epochs= 1 lstm_units= 200 embedding_dropout= 0.3 lstm_dropout= 0.5 activation_function= hard_sigmoid optimizer= nadam dense_dropout= 0.3 second_layer= False dense_units= 150 validation= 0.0 batch_size= 5 variational_dropout= False
F1-score= 40.9 Precision= 46.91 Recall=36.26 epochs= 2 lstm_units= 200 embedding_dropout= 0.3 lstm_dropout= 0.5 activation_function= hard_sigmoid optimizer= nadam dense_dropout= 0.3 second_layer= False dense_units= 150 validation= 0.0 batch_size= 5 variational_dropout= False

Epoch 1/1
228/228 [============================

...

228/228 [==============================] - 9s 39ms/step
F1-score= 27.77 Precision= 31.75 Recall=24.68 epochs= 1 lstm_units= 200 embedding_dropout= 0.3 lstm_dropout= 0.5 activation_function= hard_sigmoid optimizer= nadam dense_dropout= 0.3 second_layer= False dense_units= 150 validation= 0.0 batch_size= 5 variational_dropout= False

Epoch 1/1
228/228 [==============================] - 1s 5ms/step
F1-score= 27.77 Precision= 31.75 Recall=24.68 epochs= 1 lstm_units= 200 embedding_dropout= 0.3 lstm_dropout= 0.5 activation_function= hard_sigmoid optimizer= nadam dense_dropout= 0.3 second_layer= False dense_units= 150 validation= 0.0 batch_size= 5 variational_dropout= False
F1-score= 38.71 Precision= 40.42 Recall=37.15 epochs= 2 lstm_units= 200 embedding_dropout= 0.3 lstm_dropout= 0.5 activation_function= hard_sigmoid optimizer= nadam dense_dropout= 0.3 second_layer= False dense_units= 150 validation= 0.0 batch_size= 5 variational_dropout= False

Epoch 1/1
228/228 [===========================

/usr/local/lib/python3.7/dist-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/usr/local/lib/python3.7/dist-packages/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


Model: "model_34"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
word_input (InputLayer)      (None, 100, 300)          0         
_________________________________________________________________
dropout_69 (Dropout)         (None, 100, 300)          0         
_________________________________________________________________
bidirectional_36 (Bidirectio (None, 100, 400)          801600    
_________________________________________________________________
time_distributed_70 (TimeDis (None, 100, 150)          60150     
_________________________________________________________________
dropout_70 (Dropout)         (None, 100, 150)          0         
_________________________________________________________________
crf_34 (CRF)                 (None, 100, 9)            1458      
Total params: 863,208
Trainable params: 863,208
Non-trainable params: 0
____________________________________________________

/tensorflow-1.15.2/python3.7/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,crf_viterbi_accuracy,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
/tensorflow-1.15.2/python3.7/keras/callbacks/callbacks.py:1042: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,crf_viterbi_accuracy,accuracy,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


228/228 [==============================] - 9s 40ms/step
F1-score= 23.16 Precision= 31.85 Recall=18.19 epochs= 1 lstm_units= 200 embedding_dropout= 0.3 lstm_dropout= 0.5 activation_function= hard_sigmoid optimizer= nadam dense_dropout= 0.3 second_layer= False dense_units= 150 validation= 0.0 batch_size= 5 variational_dropout= False

Epoch 1/1
228/228 [==============================] - 1s 5ms/step
F1-score= 23.16 Precision= 31.85 Recall=18.19 epochs= 1 lstm_units= 200 embedding_dropout= 0.3 lstm_dropout= 0.5 activation_function= hard_sigmoid optimizer= nadam dense_dropout= 0.3 second_layer= False dense_units= 150 validation= 0.0 batch_size= 5 variational_dropout= False
F1-score= 41.8 Precision= 52.92 Recall=34.54 epochs= 2 lstm_units= 200 embedding_dropout= 0.3 lstm_dropout= 0.5 activation_function= hard_sigmoid optimizer= nadam dense_dropout= 0.3 second_layer= False dense_units= 150 validation= 0.0 batch_size= 5 variational_dropout= False

Epoch 1/1
228/228 [============================

In [ ]:

X_embeddings= X_fasttext 
embedding_name=X_fasttext_name

random_split=False
use_features=True 
character_embedding =Char_type("NONE")


if (use_features and character_embedding  in [Char_type.LSTM, Char_type.CNN]):
  X_train, X_test, Y_train, Y_test, X_char_tr, X_char_te, X_fea_tr, X_fea_te, emb_size=split(X_embeddings)

elif (use_features):
  X_train, X_test, Y_train, Y_test, X_fea_tr, X_fea_te, emb_size=split(X_embeddings)

elif (character_embedding  in [Char_type.LSTM, Char_type.CNN]):
  X_train, X_test, Y_train, Y_test, X_char_tr, X_char_te, emb_size=split(X_embeddings)

else:
  X_train, X_test, Y_train, Y_test, emb_size=split(X_embeddings)


lstm_units=200 #500
activation_function="hard_sigmoid" # relu
embedding_dropout=0.3
lstm_dropout=0.5
optimizer="nadam"
dense_dropout=0.3 
second_layer=False
dense_units=150
val=0.0
batch_size=5
epochs=1
variational_dropout= False

for cc in range(6):
    df1= experiment(40, False, "FastText_Fea", epochs) ## experiment ( number of epochs, repeate runs or increase epochs , name of csv file)


X_embedding  (909, 100, 300)
Y_oneHot  (909, 100, 9)
X_train  (681, 100, 300)
X_test  (228, 100, 300)
Y_train  (681, 100, 9)
Y_test  (228, 100, 9)
X_fea_tr  (681, 100, 123)
X_fea_te  (228, 100, 123)


/usr/local/lib/python3.7/dist-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/usr/local/lib/python3.7/dist-packages/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


Model: "model_35"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
word_input (InputLayer)         (None, 100, 300)     0                                            
__________________________________________________________________________________________________
feature_input (InputLayer)      (None, 100, 123)     0                                            
__________________________________________________________________________________________________
concatenate_25 (Concatenate)    (None, 100, 423)     0           word_input[0][0]                 
                                                                 feature_input[0][0]              
__________________________________________________________________________________________________
dropout_71 (Dropout)            (None, 100, 423)     0           concatenate_25[0][0]      

/tensorflow-1.15.2/python3.7/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,crf_viterbi_accuracy,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
/tensorflow-1.15.2/python3.7/keras/callbacks/callbacks.py:1042: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,crf_viterbi_accuracy,accuracy,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


228/228 [==============================] - 9s 42ms/step
F1-score= 35.55 Precision= 43.29 Recall=30.15 epochs= 1 lstm_units= 200 embedding_dropout= 0.3 lstm_dropout= 0.5 activation_function= hard_sigmoid optimizer= nadam dense_dropout= 0.3 second_layer= False dense_units= 150 validation= 0.0 batch_size= 5 variational_dropout= False

Epoch 1/1
228/228 [==============================] - 1s 5ms/step
F1-score= 35.55 Precision= 43.29 Recall=30.15 epochs= 1 lstm_units= 200 embedding_dropout= 0.3 lstm_dropout= 0.5 activation_function= hard_sigmoid optimizer= nadam dense_dropout= 0.3 second_layer= False dense_units= 150 validation= 0.0 batch_size= 5 variational_dropout= False
F1-score= 45.95 Precision= 54.19 Recall=39.89 epochs= 2 lstm_units= 200 embedding_dropout= 0.3 lstm_dropout= 0.5 activation_function= hard_sigmoid optimizer= nadam dense_dropout= 0.3 second_layer= False dense_units= 150 validation= 0.0 batch_size= 5 variational_dropout= False

Epoch 1/1
228/228 [===========================

...

228/228 [==============================] - 11s 47ms/step
F1-score= 31.01 Precision= 32.97 Recall=29.26 epochs= 1 lstm_units= 200 embedding_dropout= 0.3 lstm_dropout= 0.5 activation_function= hard_sigmoid optimizer= nadam dense_dropout= 0.3 second_layer= False dense_units= 150 validation= 0.0 batch_size= 5 variational_dropout= False

Epoch 1/1
228/228 [==============================] - 1s 5ms/step
F1-score= 31.01 Precision= 32.97 Recall=29.26 epochs= 1 lstm_units= 200 embedding_dropout= 0.3 lstm_dropout= 0.5 activation_function= hard_sigmoid optimizer= nadam dense_dropout= 0.3 second_layer= False dense_units= 150 validation= 0.0 batch_size= 5 variational_dropout= False
F1-score= 49.06 Precision= 58.15 Recall=42.43 epochs= 2 lstm_units= 200 embedding_dropout= 0.3 lstm_dropout= 0.5 activation_function= hard_sigmoid optimizer= nadam dense_dropout= 0.3 second_layer= False dense_units= 150 validation= 0.0 batch_size= 5 variational_dropout= False

Epoch 1/1
228/228 [==========================

/usr/local/lib/python3.7/dist-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/usr/local/lib/python3.7/dist-packages/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


Model: "model_40"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
word_input (InputLayer)         (None, 100, 300)     0                                            
__________________________________________________________________________________________________
feature_input (InputLayer)      (None, 100, 123)     0                                            
__________________________________________________________________________________________________
concatenate_30 (Concatenate)    (None, 100, 423)     0           word_input[0][0]                 
                                                                 feature_input[0][0]              
__________________________________________________________________________________________________
dropout_81 (Dropout)            (None, 100, 423)     0           concatenate_30[0][0]      

/tensorflow-1.15.2/python3.7/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,crf_viterbi_accuracy,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
/tensorflow-1.15.2/python3.7/keras/callbacks/callbacks.py:1042: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,crf_viterbi_accuracy,accuracy,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


228/228 [==============================] - 11s 47ms/step
F1-score= 29.41 Precision= 29.11 Recall=29.71 epochs= 1 lstm_units= 200 embedding_dropout= 0.3 lstm_dropout= 0.5 activation_function= hard_sigmoid optimizer= nadam dense_dropout= 0.3 second_layer= False dense_units= 150 validation= 0.0 batch_size= 5 variational_dropout= False

Epoch 1/1
228/228 [==============================] - 1s 6ms/step
F1-score= 29.41 Precision= 29.11 Recall=29.71 epochs= 1 lstm_units= 200 embedding_dropout= 0.3 lstm_dropout= 0.5 activation_function= hard_sigmoid optimizer= nadam dense_dropout= 0.3 second_layer= False dense_units= 150 validation= 0.0 batch_size= 5 variational_dropout= False
F1-score= 49.6 Precision= 51.65 Recall=47.71 epochs= 2 lstm_units= 200 embedding_dropout= 0.3 lstm_dropout= 0.5 activation_function= hard_sigmoid optimizer= nadam dense_dropout= 0.3 second_layer= False dense_units= 150 validation= 0.0 batch_size= 5 variational_dropout= False

Epoch 1/1
228/228 [===========================

In [ ]:

X_embeddings= X_fasttext 
embedding_name=X_fasttext_name

random_split=False
use_features=False 
character_embedding =Char_type("CNN")


if (use_features and character_embedding  in [Char_type.LSTM, Char_type.CNN]):
  X_train, X_test, Y_train, Y_test, X_char_tr, X_char_te, X_fea_tr, X_fea_te, emb_size=split(X_embeddings)

elif (use_features):
  X_train, X_test, Y_train, Y_test, X_fea_tr, X_fea_te, emb_size=split(X_embeddings)

elif (character_embedding  in [Char_type.LSTM, Char_type.CNN]):
  X_train, X_test, Y_train, Y_test, X_char_tr, X_char_te, emb_size=split(X_embeddings)

else:
  X_train, X_test, Y_train, Y_test, emb_size=split(X_embeddings)


lstm_units=200 #500
activation_function="hard_sigmoid" # relu
embedding_dropout=0.3
lstm_dropout=0.5
optimizer="nadam"
dense_dropout=0.3 
second_layer=False
dense_units=150
val=0.0
batch_size=5
epochs=1
variational_dropout= False

for cc in range(2):
    df1= experiment(40, False, "FastText_Char", epochs) ## experiment ( number of epochs, repeate runs or increase epochs , name of csv file)


X_embedding  (909, 100, 300)
Y_oneHot  (909, 100, 9)
X_train  (681, 100, 300)
X_test  (228, 100, 300)
Y_train  (681, 100, 9)
Y_test  (228, 100, 9)
X_char_tr  (681, 100, 16)
X_char_tr  (228, 100, 16)
Instructions for updating:
If using Keras pass *_constraint arguments to layers.



/usr/local/lib/python3.7/dist-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/usr/local/lib/python3.7/dist-packages/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, 100, 16, 1)   0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, 100, 16, 1, 2 3450        char_input[0][0]                 
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, 100, 16, 1, 2 5650        char_embedding[0][0]             
__________________________________________________________________________________________________
time_distributed_2 (TimeDistrib (None, 100, 16, 1, 2 0           time_distributed_1[0][0]        

/tensorflow-1.15.2/python3.7/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,crf_viterbi_accuracy,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
/tensorflow-1.15.2/python3.7/keras/callbacks/callbacks.py:1042: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,crf_viterbi_accuracy,accuracy,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


228/228 [==============================] - 2s 7ms/step
F1-score= 38.62 Precision= 42.8 Recall=35.18 epochs= 1 lstm_units= 200 embedding_dropout= 0.3 lstm_dropout= 0.5 activation_function= hard_sigmoid optimizer= nadam dense_dropout= 0.3 second_layer= False dense_units= 150 validation= 0.0 batch_size= 5 variational_dropout= False

Epoch 1/1
228/228 [==============================] - 1s 5ms/step
F1-score= 38.62 Precision= 42.8 Recall=35.18 epochs= 1 lstm_units= 200 embedding_dropout= 0.3 lstm_dropout= 0.5 activation_function= hard_sigmoid optimizer= nadam dense_dropout= 0.3 second_layer= False dense_units= 150 validation= 0.0 batch_size= 5 variational_dropout= False
F1-score= 51.04 Precision= 50.5 Recall=51.59 epochs= 2 lstm_units= 200 embedding_dropout= 0.3 lstm_dropout= 0.5 activation_function= hard_sigmoid optimizer= nadam dense_dropout= 0.3 second_layer= False dense_units= 150 validation= 0.0 batch_size= 5 variational_dropout= False

Epoch 1/1
228/228 [==============================]

/usr/local/lib/python3.7/dist-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/usr/local/lib/python3.7/dist-packages/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, 100, 16, 1)   0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, 100, 16, 1, 2 3450        char_input[0][0]                 
__________________________________________________________________________________________________
time_distributed_5 (TimeDistrib (None, 100, 16, 1, 2 5650        char_embedding[0][0]             
__________________________________________________________________________________________________
time_distributed_6 (TimeDistrib (None, 100, 16, 1, 2 0           time_distributed_5[0][0]         
____________________________________________________________________________________________

/tensorflow-1.15.2/python3.7/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,crf_viterbi_accuracy,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
/tensorflow-1.15.2/python3.7/keras/callbacks/callbacks.py:1042: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,crf_viterbi_accuracy,accuracy,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


228/228 [==============================] - 2s 7ms/step
F1-score= 24.83 Precision= 29.48 Recall=21.44 epochs= 1 lstm_units= 200 embedding_dropout= 0.3 lstm_dropout= 0.5 activation_function= hard_sigmoid optimizer= nadam dense_dropout= 0.3 second_layer= False dense_units= 150 validation= 0.0 batch_size= 5 variational_dropout= False

Epoch 1/1
228/228 [==============================] - 1s 5ms/step
F1-score= 24.83 Precision= 29.48 Recall=21.44 epochs= 1 lstm_units= 200 embedding_dropout= 0.3 lstm_dropout= 0.5 activation_function= hard_sigmoid optimizer= nadam dense_dropout= 0.3 second_layer= False dense_units= 150 validation= 0.0 batch_size= 5 variational_dropout= False
F1-score= 42.91 Precision= 55.14 Recall=35.11 epochs= 2 lstm_units= 200 embedding_dropout= 0.3 lstm_dropout= 0.5 activation_function= hard_sigmoid optimizer= nadam dense_dropout= 0.3 second_layer= False dense_units= 150 validation= 0.0 batch_size= 5 variational_dropout= False

Epoch 1/1
228/228 [============================

In [ ]:

X_embeddings= X_fasttext 
embedding_name=X_fasttext_name

random_split=False
use_features=True 
character_embedding =Char_type("CNN")


if (use_features and character_embedding  in [Char_type.LSTM, Char_type.CNN]):
  X_train, X_test, Y_train, Y_test, X_char_tr, X_char_te, X_fea_tr, X_fea_te, emb_size=split(X_embeddings)

elif (use_features):
  X_train, X_test, Y_train, Y_test, X_fea_tr, X_fea_te, emb_size=split(X_embeddings)

elif (character_embedding  in [Char_type.LSTM, Char_type.CNN]):
  X_train, X_test, Y_train, Y_test, X_char_tr, X_char_te, emb_size=split(X_embeddings)

else:
  X_train, X_test, Y_train, Y_test, emb_size=split(X_embeddings)


lstm_units=200 #500
activation_function="hard_sigmoid" # relu
embedding_dropout=0.3
lstm_dropout=0.5
optimizer="nadam"
dense_dropout=0.3 
second_layer=False
dense_units=150
val=0.0
batch_size=5
epochs=1
variational_dropout= False

for cc in range(4):
    df1= experiment(40, False, "FastText_Fea_Char", epochs) ## experiment ( number of epochs, repeate runs or increase epochs , name of csv file)


X_embedding  (909, 100, 300)
Y_oneHot  (909, 100, 9)
X_train  (681, 100, 300)
X_test  (228, 100, 300)
Y_train  (681, 100, 9)
Y_test  (228, 100, 9)
X_char_tr  (681, 100, 16)
X_char_tr  (228, 100, 16)
X_fea_tr  (681, 100, 123)
X_fea_te  (228, 100, 123)


/usr/local/lib/python3.7/dist-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/usr/local/lib/python3.7/dist-packages/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, 100, 16, 1)   0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, 100, 16, 1, 2 3450        char_input[0][0]                 
__________________________________________________________________________________________________
time_distributed_9 (TimeDistrib (None, 100, 16, 1, 2 5650        char_embedding[0][0]             
__________________________________________________________________________________________________
word_input (InputLayer)         (None, 100, 300)     0                                            
____________________________________________________________________________________________

/tensorflow-1.15.2/python3.7/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,crf_viterbi_accuracy,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
/tensorflow-1.15.2/python3.7/keras/callbacks/callbacks.py:1042: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,crf_viterbi_accuracy,accuracy,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


228/228 [==============================] - 2s 8ms/step
F1-score= 21.58 Precision= 39.69 Recall=14.82 epochs= 1 lstm_units= 200 embedding_dropout= 0.3 lstm_dropout= 0.5 activation_function= hard_sigmoid optimizer= nadam dense_dropout= 0.3 second_layer= False dense_units= 150 validation= 0.0 batch_size= 5 variational_dropout= False

Epoch 1/1
228/228 [==============================] - 1s 5ms/step
F1-score= 21.58 Precision= 39.69 Recall=14.82 epochs= 1 lstm_units= 200 embedding_dropout= 0.3 lstm_dropout= 0.5 activation_function= hard_sigmoid optimizer= nadam dense_dropout= 0.3 second_layer= False dense_units= 150 validation= 0.0 batch_size= 5 variational_dropout= False
F1-score= 51.49 Precision= 51.84 Recall=51.15 epochs= 2 lstm_units= 200 embedding_dropout= 0.3 lstm_dropout= 0.5 activation_function= hard_sigmoid optimizer= nadam dense_dropout= 0.3 second_layer= False dense_units= 150 validation= 0.0 batch_size= 5 variational_dropout= False

Epoch 1/1
228/228 [============================

...

228/228 [==============================] - 2s 10ms/step
F1-score= 42.21 Precision= 49.66 Recall=36.7 epochs= 1 lstm_units= 200 embedding_dropout= 0.3 lstm_dropout= 0.5 activation_function= hard_sigmoid optimizer= nadam dense_dropout= 0.3 second_layer= False dense_units= 150 validation= 0.0 batch_size= 5 variational_dropout= False

Epoch 1/1
228/228 [==============================] - 1s 5ms/step
F1-score= 42.21 Precision= 49.66 Recall=36.7 epochs= 1 lstm_units= 200 embedding_dropout= 0.3 lstm_dropout= 0.5 activation_function= hard_sigmoid optimizer= nadam dense_dropout= 0.3 second_layer= False dense_units= 150 validation= 0.0 batch_size= 5 variational_dropout= False
F1-score= 49.45 Precision= 55.6 Recall=44.53 epochs= 2 lstm_units= 200 embedding_dropout= 0.3 lstm_dropout= 0.5 activation_function= hard_sigmoid optimizer= nadam dense_dropout= 0.3 second_layer= False dense_units= 150 validation= 0.0 batch_size= 5 variational_dropout= False

Epoch 1/1
228/228 [==============================

/usr/local/lib/python3.7/dist-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/usr/local/lib/python3.7/dist-packages/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, 100, 16, 1)   0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, 100, 16, 1, 2 3450        char_input[0][0]                 
__________________________________________________________________________________________________
time_distributed_21 (TimeDistri (None, 100, 16, 1, 2 5650        char_embedding[0][0]             
__________________________________________________________________________________________________
word_input (InputLayer)         (None, 100, 300)     0                                            
____________________________________________________________________________________________

/tensorflow-1.15.2/python3.7/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,crf_viterbi_accuracy,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
/tensorflow-1.15.2/python3.7/keras/callbacks/callbacks.py:1042: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,crf_viterbi_accuracy,accuracy,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


228/228 [==============================] - 2s 11ms/step
F1-score= 38.26 Precision= 46.74 Recall=32.38 epochs= 1 lstm_units= 200 embedding_dropout= 0.3 lstm_dropout= 0.5 activation_function= hard_sigmoid optimizer= nadam dense_dropout= 0.3 second_layer= False dense_units= 150 validation= 0.0 batch_size= 5 variational_dropout= False

Epoch 1/1
228/228 [==============================] - 1s 5ms/step
F1-score= 38.26 Precision= 46.74 Recall=32.38 epochs= 1 lstm_units= 200 embedding_dropout= 0.3 lstm_dropout= 0.5 activation_function= hard_sigmoid optimizer= nadam dense_dropout= 0.3 second_layer= False dense_units= 150 validation= 0.0 batch_size= 5 variational_dropout= False
F1-score= 46.19 Precision= 41.28 Recall=52.42 epochs= 2 lstm_units= 200 embedding_dropout= 0.3 lstm_dropout= 0.5 activation_function= hard_sigmoid optimizer= nadam dense_dropout= 0.3 second_layer= False dense_units= 150 validation= 0.0 batch_size= 5 variational_dropout= False

Epoch 1/1
228/228 [===========================

In [ ]:
## examine mispredictions

#X_arabert, X_arabert_name= arabert(X_words, "aubmindlab/bert-large-arabertv02", "all", True,  "mean", False, False,1024 )

X_embeddings= X_arabert 
embedding_name=X_arabert_name
X_words_te=X_words[681:]  ## 681 is number of training sentences
random_split=False
use_features=False
character_embedding =Char_type("NONE")
if (use_features and character_embedding  in [Char_type.LSTM, Char_type.CNN]):
  X_train, X_test, Y_train, Y_test, X_char_tr, X_char_te, X_fea_tr, X_fea_te, emb_size=split(X_embeddings)

elif (use_features):
  X_train, X_test, Y_train, Y_test, X_fea_tr, X_fea_te, emb_size=split(X_embeddings)

elif (character_embedding  in [Char_type.LSTM, Char_type.CNN]):
  X_train, X_test, Y_train, Y_test, X_char_tr, X_char_te, emb_size=split(X_embeddings)

else:
  X_train, X_test, Y_train, Y_test, emb_size=split(X_embeddings)



model= load_model("/content/drive/MyDrive/Ali lotfy/saved models AQMAR/AraBert 79.48/weights.h5", "/content/drive/MyDrive/Ali lotfy/saved models AQMAR/AraBert 79.48/params.json")
#model= load_model("/content/drive/MyDrive/Ali lotfy/saved models/Arabert 10 88.06/weights.h5", "/content/drive/MyDrive/Ali lotfy/saved models/Arabert 10 88.06/params.json")

model.summary()

test_pred = model.predict(([np.array(X_test), np.array(X_fea_te) ]), verbose=1)  
#test_pred = model.predict(np(X_test), verbose=1)   

pred_labels = pred2label(test_pred)
test_labels = pred2label(Y_test)

print(classification_report(test_labels, pred_labels, digits=4))
print("F1: ",f1_score(test_labels, pred_labels))
print("precision: ",precision_score(test_labels, pred_labels))
print("recall: ",recall_score(test_labels, pred_labels))



X_embedding  (909, 100, 1024)
Y_oneHot  (909, 100, 9)
X_train  (681, 100, 1024)
X_test  (228, 100, 1024)
Y_train  (681, 100, 9)
Y_test  (228, 100, 9)
Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
word_input (InputLayer)         (None, 100, 1024)    0                                            
__________________________________________________________________________________________________
feature_input (InputLayer)      (None, 100, 123)     0                                            
__________________________________________________________________________________________________
concatenate_5 (Concatenate)     (None, 100, 1147)    0           word_input[0][0]                 
                                                                 feature_input[0][0]              
________________________________________

In [ ]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

print(classification_report( test_labels, pred_labels, digits=4))
print("F1: ",f1_score( test_labels, pred_labels))
print("precision: ",precision_score( test_labels,  pred_labels))
print("recall: ",recall_score( test_labels,  pred_labels))

              precision    recall  f1-score   support

         LOC     0.8857    0.8558    0.8705       326
        MISC     0.6710    0.6413    0.6558       722
         ORG     0.6942    0.8235    0.7534       102
        PERS     0.9221    0.8939    0.9078       424

   micro avg     0.7840    0.7656    0.7747      1574
   macro avg     0.7933    0.8036    0.7969      1574
weighted avg     0.7846    0.7656    0.7745      1574

F1:  0.7746705239472838
precision:  0.7839947950553026
recall:  0.7655654383735705


In [ ]:
y_true = np.array(test_labels).reshape([-1])
y_pred = np.array(pred_labels).reshape([-1])

In [ ]:
len(y_true)

19456

In [ ]:
from sklearn.metrics import f1_score , classification_report , recall_score, precision_score


In [ ]:
labels = ['I-LOC', 'I-MISC', 'B-MISC','B-ORG', 'B-PERS', 'B-LOC', 'I-ORG', 'I-PERS', 'O']
labels.remove('O') # remove 'O' label from evaluation
sorted_labels = sorted(labels,key=lambda name: (name[1:], name[0])) # group B and I results
#print(sklearn_crfsuite.metrics.flat_classification_report(y_test, y_pred, labels=sorted_labels, digits=3))

In [ ]:
sorted_labels

['B-LOC', 'I-LOC', 'B-MISC', 'I-MISC', 'B-ORG', 'I-ORG', 'B-PERS', 'I-PERS']

In [ ]:
target_names = ['I-LOC', 'I-MISC', 'B-MISC','B-ORG', 'B-PERS', 'B-LOC', 'I-ORG', 'I-PERS', 'O']
#print(classification_report(pred_labels, test_labels, target_names=target_names))
print(classification_report(y_true, y_pred, labels=sorted_labels, digits=4))

              precision    recall  f1-score   support

       B-LOC     0.8952    0.8677    0.8812       325
       I-LOC     0.8903    0.8846    0.8875       156
      B-MISC     0.7865    0.7462    0.7658       721
      I-MISC     0.7506    0.5996    0.6667       517
       B-ORG     0.7672    0.8725    0.8165       102
       I-ORG     0.8512    0.8306    0.8408       124
      B-PERS     0.9364    0.9033    0.9196       424
      I-PERS     0.9755    0.9636    0.9695       330

   micro avg     0.8511    0.8007    0.8251      2699
   macro avg     0.8566    0.8335    0.8435      2699
weighted avg     0.8476    0.8007    0.8222      2699



In [ ]:
###================ evaluation method ===============
#this method evaluate all NEs, hocwver results are different by a small margin

def evaluate(test_labels, pred_labels, entity_class, X_words_te):
  i=0 # loop on sentences
  TP=[0,0,0,0]
  FN=[0,0,0,0]
  FP=[0,0,0,0]
  FPC=[0,0,0,0] # FPC is entities that are not FP, they should equal TP but they are not as there is a condition that is not handeled
               #to do : add the difference between FPC and TP to FP.

  classes = ['PERS', 'LOC', 'ORG', 'MISC']
  i=0
  count= 0
  print("{:20} {:10} {:10} {:10}".format("Word", "True", "Pred",  "sentence"))
 
  while i < len(test_labels):
    j=0 # loop on tokens
    while j < len (test_labels[i]):
      
      if(entity_class in test_labels[i][j]):
        entity_start=j
        class_name= test_labels[i][j] [2:]
        while (True):
          if (test_labels[i][j] != pred_labels[i][j]):
            j+=1
            FN[classes.index(class_name)]+=1
            show="allTrue"
            count+=1
            break;
          elif class_name not in test_labels[i][j+1] or 'B-' in test_labels[i][j+1] :
            if class_name in pred_labels[i][j+1] and 'B-' not in pred_labels[i][j+1]: 
              FN[classes.index(class_name)]+=1
              show="allTrue+"
              count+=1
            else:
              TP[classes.index(class_name)]+=1
              show=""
            j+=1
            break;
          else:
            j+=1
        show_mispredictions(entity_start, show, entity_class, i, test_labels, pred_labels, X_words_te) # show mispredictions
      else:
        j+=1
    i+=1
  print(count)

  i=0
  count=0
  print("{:20} {:10} {:10} {:10}".format("Word", "True", "Pred",  "sentence"))

  while i < len(pred_labels):
    j=0 # loop on tokens
    while j < len (pred_labels[i]):
      if(entity_class in pred_labels[i][j]):
        entity_start=j
        class_name= pred_labels[i][j] [2:]
        while (True):
          if (pred_labels[i][j] != test_labels[i][j]):
            j+=1
            FP[classes.index(class_name)]+=1
            show="allPred"
            count+=1
            break;
          elif class_name not in pred_labels[i][j+1] or 'B-' in pred_labels[i][j+1]:
            if class_name in test_labels[i][j+1] and 'B-' not in test_labels[i][j+1]: 
              FP[classes.index(class_name)]+=1
              show="allPred+"
              count+=1
            else:
              FPC[classes.index(class_name)]+=1
              show=""
            j+=1

            break;
          else:
            j+=1
        show_mispredictions(entity_start, show, entity_class, i, test_labels, pred_labels, X_words_te) # show mispredictions
      else:
        j+=1
    i+=1
  print(count)
  print('TP: ',TP)
  print('FN: ',FN)
  print('FP: ',FP)
  print('FPC: ',FPC)



def show_mispredictions( entity_start, show, entity_class, i, test_labels, pred_labels, X_words_te):
  if show !="":
    if "True" in show:
      series = test_labels
    else:
      series = pred_labels
    print("{:20} {:10} {:10} [{}]".format(X_words_te[i][entity_start],test_labels[i][entity_start],pred_labels[i][entity_start], i))
    entity_start+=1
    try:
      while("I-" in series[i][entity_start]):
        print("{:20} {:10} {:10} [{}]".format(X_words_te[i][entity_start],test_labels[i][entity_start],pred_labels[i][entity_start], i))
        entity_start+=1
      if "+" in show:
        print("{:20} {:10} {:10} [{}]".format(X_words_te[i][entity_start],test_labels[i][entity_start],pred_labels[i][entity_start], i))
    except:
      filler=0
    print("-------------------------------------------------------------------------")




In [ ]:
evaluate(test_labels, pred_labels, "B-", X_words_te)

Word                 True       Pred       sentence  
ونايفة               B-LOC      O          [0]
-------------------------------------------------------------------------
الولايات             B-LOC      B-LOC      [1]
المتحدة              B-ORG      I-LOC      [1]
-------------------------------------------------------------------------
المتحدة              B-ORG      I-LOC      [1]
-------------------------------------------------------------------------
تراب                 B-PERS     O          [1]
-------------------------------------------------------------------------
الأمن                B-ORG      O          [1]
-------------------------------------------------------------------------
دار                  B-ORG      O          [2]
الهيئة               I-ORG      O          [2]
المصرية              I-ORG      O          [2]
العامة               I-ORG      O          [2]
-------------------------------------------------------------------------
التأسلم              B-MISC     

In [ ]:
#X_words_te=X_words[257:]  ## 257 is number of training sentences
evaluate(test_labels, pred_labels, "B-PERS", X_words_te)

Word                 True       Pred       sentence  
تراب                 B-PERS     O          [1]
-------------------------------------------------------------------------
حظر                  B-PERS     O          [3]
مسعود                I-PERS     B-PERS     [3]
البارزاني            I-PERS     I-PERS     [3]
-------------------------------------------------------------------------
لاريجاني             B-PERS     I-PERS     [5]
-------------------------------------------------------------------------
البابا               B-PERS     O          [5]
بنديكت               I-PERS     B-PERS     [5]
ال                   I-PERS     O          [5]
-------------------------------------------------------------------------
الظواهري             B-PERS     B-PERS     [6]
البابا               I-PERS     O          [6]
بنديكت               I-PERS     B-PERS     [6]
ال                   I-PERS     O          [6]
-------------------------------------------------------------------------
عمر         

In [ ]:
#X_words_te=X_words[257:]  ## 257 is number of training sentences
evaluate(test_labels, pred_labels, "B-LOC", X_words_te)

Word                 True       Pred       sentence  
ونايفة               B-LOC      O          [0]
-------------------------------------------------------------------------
الولايات             B-LOC      B-LOC      [1]
المتحدة              B-ORG      I-LOC      [1]
-------------------------------------------------------------------------
كردستان              B-LOC      B-LOC      [3]
العراق               I-LOC      B-LOC      [3]
-------------------------------------------------------------------------
بحر                  B-LOC      O          [5]
الصين                I-LOC      B-LOC      [5]
-------------------------------------------------------------------------
لخالد                B-LOC      B-PERS     [6]
-------------------------------------------------------------------------
قطرية                B-LOC      O          [7]
-------------------------------------------------------------------------
سيرا                 B-LOC      B-LOC      [8]
دو                   I-LOC      

In [ ]:
#X_words_te=X_words[257:]  ## 257 is number of training sentences
evaluate(test_labels, pred_labels, "B-ORG", X_words_te)

Word                 True       Pred       sentence  
المتحدة              B-ORG      I-LOC      [1]
-------------------------------------------------------------------------
الأمن                B-ORG      O          [1]
-------------------------------------------------------------------------
دار                  B-ORG      O          [2]
الهيئة               I-ORG      O          [2]
المصرية              I-ORG      O          [2]
العامة               I-ORG      O          [2]
-------------------------------------------------------------------------
حزب                  B-ORG      O          [2]
التجمع               I-ORG      O          [2]
المصري               I-ORG      O          [2]
-------------------------------------------------------------------------
قناة                 B-ORG      O          [2]
السويس               I-ORG      I-ORG      [2]
الاستعمارية          I-ORG      O          [2]
-------------------------------------------------------------------------
الشرق       

In [ ]:
#X_words_te=X_words[257:]  ## 257 is number of training sentences
evaluate(test_labels, pred_labels, "B-MISC", X_words_te)

Word                 True       Pred       sentence  
التأسلم              B-MISC     O          [2]
السياسي              I-MISC     O          [2]
-------------------------------------------------------------------------
الخلفاء              B-MISC     O          [2]
-------------------------------------------------------------------------
الراشدين             B-MISC     O          [2]
-------------------------------------------------------------------------
الدولة               B-MISC     B-ORG      [2]
الأموية              I-MISC     I-ORG      [2]
-------------------------------------------------------------------------
العباسية             B-MISC     O          [2]
-------------------------------------------------------------------------
بوصوله               B-MISC     O          [5]
-------------------------------------------------------------------------
المسيحية             B-MISC     O          [6]
-------------------------------------------------------------------------
مانوس

In [ ]:
##  Person

#=== False negative=========
count=0
i=0
added=[]
window=3
print("{:20} {:10} {:10} {:10} {}".format("Word", "True", "Pred",  "sentence", "contetx"))
while(i<np.shape(pred_labels)[0]):
  j=0
  pred= pred_labels[i]
  true=test_labels[i]
  word=X_words_te[i]
  while(j<sent_len):
    if(j<window):
      context=window
    elif(j+window>=sent_len):
      context=sent_len-3
    else:
      context= j

    subCounter=1
    if(pred[j] != true[j] and true[j] in ['B-PERS', 'I-PERS']) or (pred[j-1]==true[j-1] and true[j-1] in ['B-PERS', 'I-PERS'] and true[j]=='O' and pred[j] in [ 'I-PERS']):
      while(True):
        if(true[j] in ['B-PERS']):
          break
        if(true[j-subCounter] in ['B-PERS'] ):
          print("{:20} {:10} {:10} [{}]          {:50}".format(word[j-subCounter],true[j-subCounter],pred[j-subCounter], i, " ".join(word[context-window: context+window])))
          break
        else:
          print("{:20} {:10} {:10} [{}]          {:50}".format(word[j-subCounter],true[j-subCounter],pred[j-subCounter], i, " ".join(word[context-window: context+window])))
          subCounter+=1

      print("{:20} {:10} {:10} [{}]          {:50}".format(word[j],true[j],pred[j], i, " ".join(word[context-window: context+window])))
      count+=1
      subCounter=1
      while(True):
        if(true[j+subCounter] in ['I-PERS']):
          print("{:20} {:10} {:10} [{}]          {:50}".format(word[j+subCounter],true[j+subCounter],pred[j+subCounter], i, " ".join(word[context-window: context+window])))
          subCounter+=1
        else:
          break

      print("-------------------------------------------------------------------------")
    j+=subCounter
  i+=1
print(count)



##  false positive

count=0
i=0
added=[]
window=3
print("{:20} {:10} {:10} {:10} {}".format("Word", "True", "Pred",  "sentence", "contetx"))
while(i<np.shape(pred_labels)[0]):
  j=0
  pred= pred_labels[i]
  true=test_labels[i]
  word=X_words_te[i]
  while(j<sent_len):
    if(j<window):
      context=window
    elif(j+window>=sent_len):
      context=sent_len-3
    else:
      context= j

    subCounter=1
    if( true[j] =='O'and pred[j] in ['B-PERS', 'I-PERS']):
      print("{:20} {:10} {:10} [{}]          {:50}".format(word[j],true[j],pred[j], i, " ".join(word[context-window: context+window])))
      count+=1

      print("-------------------------------------------------------------------------")
    j+=subCounter
  i+=1
print(count)

Word                 True       Pred       sentence   contetx
تراب                 B-PERS     O          [1]          الاستقرار في كامل تراب البلاد ،                   
-------------------------------------------------------------------------
حظر                  B-PERS     O          [3]          قال فيه إن حظر مسعود البارزاني                    
مسعود                I-PERS     B-PERS     [3]          قال فيه إن حظر مسعود البارزاني                    
البارزاني            I-PERS     I-PERS     [3]          قال فيه إن حظر مسعود البارزاني                    
-------------------------------------------------------------------------
لاريجاني             B-PERS     I-PERS     [5]          خافيير سولانا وعلي لاريجاني في برلين              
-------------------------------------------------------------------------
البابا               B-PERS     O          [5]          موقفه من تصريحات البابا بنديكت ال                 
بنديكت               I-PERS     B-PERS     [5]          موقفه من تصريحات ا

In [ ]:
# Location

count=0
i=0
added=[]
window=3
print("{:20} {:10} {:10} {:10} {}".format("Word", "True", "Pred",  "sentence", "contetx"))
while(i<np.shape(pred_labels)[0]):
  j=0
  pred= pred_labels[i]
  true=test_labels[i]
  word=X_words_te[i]
  while(j<sent_len):
    if(j<window):
      context=window
    elif(j+window>=sent_len):
      context=sent_len-3
    else:
      context= j

    subCounter=1
    if(pred[j] != true[j] and true[j] in ['B-LOC', 'I-LOC']):
      while(True):
        if(true[j] in ['B-LOC']):
          break
        if(true[j-subCounter] in ['B-LOC'] ):
          print("{:20} {:10} {:10} [{}]          {:50}".format(word[j-subCounter],true[j-subCounter],pred[j-subCounter], i, " ".join(word[context-window: context+window])))
          break
        else:
          print("{:20} {:10} {:10} [{}]          {:50}".format(word[j-subCounter],true[j-subCounter],pred[j-subCounter], i, " ".join(word[context-window: context+window])))
          subCounter+=1

      print("{:20} {:10} {:10} [{}]          {:50}".format(word[j],true[j],pred[j], i, " ".join(word[context-window: context+window])))
      count+=1
      subCounter=1
      while(True):
        if(true[j+subCounter] in ['I-LOC']):
          print("{:20} {:10} {:10} [{}]          {:50}".format(word[j+subCounter],true[j+subCounter],pred[j+subCounter], i, " ".join(word[context-window: context+window])))
          subCounter+=1
        else:
          break

      print("-------------------------------------------------------------------------")
    j+=subCounter
  i+=1
print(count)


##  false positive

count=0
i=0
added=[]
window=3
print("{:20} {:10} {:10} {:10} {}".format("Word", "True", "Pred",  "sentence", "contetx"))
while(i<np.shape(pred_labels)[0]):
  j=0
  pred= pred_labels[i]
  true=test_labels[i]
  word=X_words_te[i]
  while(j<sent_len):
    if(j<window):
      context=window
    elif(j+window>=sent_len):
      context=sent_len-3
    else:
      context= j

    subCounter=1
    if( true[j] =='O'and pred[j] in ['B-LOC', 'I-LOC']):
      print("{:20} {:10} {:10} [{}]          {:50}".format(word[j],true[j],pred[j], i, " ".join(word[context-window: context+window])))
      count+=1

      print("-------------------------------------------------------------------------")
    j+=subCounter
  i+=1
print(count)

In [ ]:
# Organization

count=0
i=0
added=[]
window=3
print("{:20} {:10} {:10} {:10} {}".format("Word", "True", "Pred",  "sentence", "contetx"))
while(i<np.shape(pred_labels)[0]):
  j=0
  pred= pred_labels[i]
  true=test_labels[i]
  word=X_words_te[i]
  while(j<sent_len):
    if(j<window):
      context=window
    elif(j+window>=sent_len):
      context=sent_len-3
    else:
      context= j

    subCounter=1
    if(pred[j] != true[j] and true[j] in ['B-ORG', 'I-ORG']):
      while(True):
        if(true[j] in ['B-ORG']):
          break
        if(true[j-subCounter] in ['B-ORG'] ):
          print("{:20} {:10} {:10} [{}]          {:50}".format(word[j-subCounter],true[j-subCounter],pred[j-subCounter], i, " ".join(word[context-window: context+window])))
          break
        else:
          print("{:20} {:10} {:10} [{}]          {:50}".format(word[j-subCounter],true[j-subCounter],pred[j-subCounter], i, " ".join(word[context-window: context+window])))
          subCounter+=1

      print("{:20} {:10} {:10} [{}]          {:50}".format(word[j],true[j],pred[j], i, " ".join(word[context-window: context+window])))
      count+=1
      subCounter=1
      while(True):
        if(true[j+subCounter] in ['I-ORG']):
          print("{:20} {:10} {:10} [{}]          {:50}".format(word[j+subCounter],true[j+subCounter],pred[j+subCounter], i, " ".join(word[context-window: context+window])))
          subCounter+=1
        else:
          break

      print("-------------------------------------------------------------------------")
    j+=subCounter
  i+=1
print(count)


##  false positive

count=0
i=0
added=[]
window=3
print("{:20} {:10} {:10} {:10} {}".format("Word", "True", "Pred",  "sentence", "contetx"))
while(i<np.shape(pred_labels)[0]):
  j=0
  pred= pred_labels[i]
  true=test_labels[i]
  word=X_words_te[i]
  while(j<sent_len):
    if(j<window):
      context=window
    elif(j+window>=sent_len):
      context=sent_len-3
    else:
      context= j

    subCounter=1
    if( true[j] =='O'and pred[j] in ['B-ORG', 'I-ORG']):
      print("{:20} {:10} {:10} [{}]          {:50}".format(word[j],true[j],pred[j], i, " ".join(word[context-window: context+window])))
      count+=1

      print("-------------------------------------------------------------------------")
    j+=subCounter
  i+=1
print(count)

Word                 True       Pred       sentence   contetx
المتحدة              B-ORG      I-LOC      [1]          حال استندت الولايات المتحدة مثلا إلى              
-------------------------------------------------------------------------
الأمن                B-ORG      O          [1]          العاصمة كابول وفوضت الأمن للمجاهدين في            
-------------------------------------------------------------------------
دار                  B-ORG      O          [2]          صدر حديثا عن دار الهيئة المصرية                   
الهيئة               I-ORG      O          [2]          صدر حديثا عن دار الهيئة المصرية                   
المصرية              I-ORG      O          [2]          صدر حديثا عن دار الهيئة المصرية                   
العامة               I-ORG      O          [2]          صدر حديثا عن دار الهيئة المصرية                   
-------------------------------------------------------------------------
حزب                  B-ORG      O          [2]          رفعت السعيد رئيس ح

In [ ]:
Word                 True       Pred       sentence   contetx
ونايفة               O          B-ORG      [0]          الصالحية ونايفة المفرق - غيث الطراونة            fail 
-------------------------------------------------------------------------
الإخوان              O          B-ORG      [2]          مقولات وأقاويل جماعة الإخوان المسلمين ،           wrong
-------------------------------------------------------------------------
المسلمين             O          I-ORG      [2]          وأقاويل جماعة الإخوان المسلمين ، فتارة            
-------------------------------------------------------------------------
الأكراد              O          B-ORG      [3]          بدءا بالتصعيد بين الأكراد والعرب وانتهاء          fail
-------------------------------------------------------------------------
وسنة                 O          B-ORG      [3]          إحداهما بين الأميركيين وسنة العراق والأخرى        fail
-------------------------------------------------------------------------
والأكراد             O          B-ORG      [3]          ثالثة بين العرب والأكراد في المناطق               fail
-------------------------------------------------------------------------
بمنظمة               O          B-ORG      [5]          التراث الإنساني الخاصة بمنظمة الأمم المتحدة       wrong
-------------------------------------------------------------------------
للتربية              O          I-ORG      [5]          بمنظمة الأمم المتحدة للتربية والعلوم والثقافة     
-------------------------------------------------------------------------
والعلوم              O          I-ORG      [5]          الأمم المتحدة للتربية والعلوم والثقافة (          
-------------------------------------------------------------------------
والثقافة             O          I-ORG      [5]          المتحدة للتربية والعلوم والثقافة ( يونيسكو        
-------------------------------------------------------------------------
حماس                 O          B-ORG      [7]          كافة بما فيها حماس والسلطة لديها            wrong      
-------------------------------------------------------------------------
بحماس                O          B-ORG      [7]          ذلك قال القيادي بحماس إسماعيل رضوان              wrong 
-------------------------------------------------------------------------
للراليات             O          I-ORG      [10]          سيارة فريق قطر للراليات متسوبيشي ال               convention
-------------------------------------------------------------------------
من                   O          B-ORG      [14]          باسم " الحزب من اجل مجتمع            wrong             
-------------------------------------------------------------------------
اجل                  O          I-ORG      [14]          " الحزب من اجل مجتمع ديمقراطي                     
-------------------------------------------------------------------------
مجتمع                O          I-ORG      [14]          الحزب من اجل مجتمع ديمقراطي "                     
-------------------------------------------------------------------------
ديمقراطي             O          I-ORG      [14]          من اجل مجتمع ديمقراطي " لوكالة                    
-------------------------------------------------------------------------
الاكراد              O          B-ORG      [14]          لدعمه المفترض للمتمردين الاكراد " اذا     fail        
-------------------------------------------------------------------------
مجلس                 O          B-ORG      [16]          على توصية من مجلس الامن وفي                   wrong    
-------------------------------------------------------------------------
الامن                O          I-ORG      [16]          توصية من مجلس الامن وفي الواقع                    wrong
-------------------------------------------------------------------------
جامعة                O          B-ORG      [17]          التاريخ الحديث في جامعة سان اندرو                 
-------------------------------------------------------------------------
الوكالة              O          B-ORG      [18]          المحافظة على دور الوكالة الدولية للطاقة    wrong       
-------------------------------------------------------------------------
الدولية              O          I-ORG      [18]          على دور الوكالة الدولية للطاقة الذرية             
-------------------------------------------------------------------------
للطاقة               O          I-ORG      [18]          دور الوكالة الدولية للطاقة الذرية والحيلولة       
-------------------------------------------------------------------------
الذرية               O          I-ORG      [18]          الوكالة الدولية للطاقة الذرية والحيلولة دون       
-------------------------------------------------------------------------
العسكر               O          B-ORG      [21]          التفجيرات بمساعدة حركة العسكر الطيبة .          wrong  
-------------------------------------------------------------------------
الطيبة               O          I-ORG      [21]          بمساعدة حركة العسكر الطيبة . وذكرت               
-------------------------------------------------------------------------
لوكالة               O          B-ORG      [21]          أوبورتشينيتي " التابعة لوكالة الفضاء الأمريكية   wrong 
-------------------------------------------------------------------------
الفضاء               O          I-ORG      [21]          " التابعة لوكالة الفضاء الأمريكية ناسا            
-------------------------------------------------------------------------
الأمريكية            O          I-ORG      [21]          التابعة لوكالة الفضاء الأمريكية ناسا صورا         
-------------------------------------------------------------------------
الرياضي              O          I-ORG      [26]          مجلس إدارة الترجي الرياضي التونسي مدرب            fail
-------------------------------------------------------------------------
النجم                O          B-ORG      [26]          من غريمه المحلي النجم الساحلي .                   wrong
-------------------------------------------------------------------------
الساحلي              O          I-ORG      [26]          غريمه المحلي النجم الساحلي . ورغم                 
-------------------------------------------------------------------------
سني                  O          B-ORG      [34]          عبد المهدي والأخر سني وهو طارق     fail               
-------------------------------------------------------------------------
الريال               O          B-ORG      [36]          أن حارس مرمى الريال كاسياس تدخل        wrong           
-------------------------------------------------------------------------
الصريح               O          B-ORG      [44]          ، اخرج فيها الصريح - احد                    wrong      
-------------------------------------------------------------------------
الصريح               O          B-ORG      [44]          شغب من جماهير الصريح احتواه رجال                 wrong 
-------------------------------------------------------------------------
البقعة               O          B-ORG      [45]          له صدها حارس البقعة ، مواصلا                      wrong
-------------------------------------------------------------------------
البقعة               O          B-ORG      [45]          والكأس ، واكد البقعة تجذر الأصرار                 wrong
-------------------------------------------------------------------------
الشباب               O          B-ORG      [45]          ابو عرقوب لاعب الشباب البطاقة الحمراء             wrong
-------------------------------------------------------------------------
الصريح               O          B-ORG      [46]          علي غرايبة طبق الصريح منذ البداية                 wrong
-------------------------------------------------------------------------
الصريح               O          B-ORG      [46]          . كان قبلها الصريح يعتمد على                      wrong
-------------------------------------------------------------------------
الصريح               O          B-ORG      [46]          الحفناوي . تخلى الصريح مع بداية                   wrong
-------------------------------------------------------------------------
الصريح               O          B-ORG      [46]          في دفاعه واصل الصريح اندفاعه وهو                  wrong
-------------------------------------------------------------------------
البقعة               O          B-ORG      [46]          سرعة من قبل البقعة الذي كشف                       wrong
-------------------------------------------------------------------------
البقعة               O          B-ORG      [46]          عددية على دفاع البقعة وأضحت تمريراتهم             wrong
-------------------------------------------------------------------------
البقعة               O          B-ORG      [47]          مقابله على ميمنة البقعة كان يحتاج                 wrong
-------------------------------------------------------------------------
فتح                  O          B-ORG      [47]          مسيرة نظمها مؤيدو فتح للتنديد بالاقتتال           wrong
-------------------------------------------------------------------------
لمجلس                O          B-ORG      [49]          يجري الأمين العام لمجلس الأمن القومي              wrong
-------------------------------------------------------------------------
الأمن                O          I-ORG      [49]          الأمين العام لمجلس الأمن القومي الروسي            
-------------------------------------------------------------------------
50

In [ ]:
# Miscellaneous

count=0
i=0
added=[]
window=3
print("{:20} {:10} {:10} {:10} {}".format("Word", "True", "Pred",  "sentence", "contetx"))
while(i<np.shape(pred_labels)[0]):
  j=0
  pred= pred_labels[i]
  true=test_labels[i]
  word=X_words_te[i]
  while(j<sent_len):
    if(j<window):
      context=window
    elif(j+window>=sent_len):
      context=sent_len-3
    else:
      context= j
    subCounter=1
    if(pred[j] != true[j] and true[j] in ['B-MISC', 'I-MISC']):
      while(True):
        if(true[j] in ['B-MISC']):
          break
        if(true[j-subCounter] in ['B-MISC'] ):
          print("{:20} {:10} {:10} [{}]          {:50}".format(word[j-subCounter],true[j-subCounter],pred[j-subCounter], i, " ".join(word[context-window: context+window])))
          break
        else:
          print("{:20} {:10} {:10} [{}]          {:50}".format(word[j-subCounter],true[j-subCounter],pred[j-subCounter], i, " ".join(word[context-window: context+window])))
          subCounter+=1

      print("{:20} {:10} {:10} [{}]          {:50}".format(word[j],true[j],pred[j], i, " ".join(word[context-window: context+window])))
      count+=1
      subCounter=1
      while(True):
        if(true[j+subCounter] in ['I-MISC']):
          print("{:20} {:10} {:10} [{}]          {:50}".format(word[j+subCounter],true[j+subCounter],pred[j+subCounter], i, " ".join(word[context-window: context+window])))
          subCounter+=1
        else:
          break

      print("-------------------------------------------------------------------------")
    j+=subCounter
  i+=1
print(count)


##  false positive

count=0
i=0
added=[]
window=3
print("{:20} {:10} {:10} {:10} {}".format("Word", "True", "Pred",  "sentence", "contetx"))
while(i<np.shape(pred_labels)[0]):
  j=0
  pred= pred_labels[i]
  true=test_labels[i]
  word=X_words_te[i]
  while(j<sent_len):
    if(j<window):
      context=window
    elif(j+window>=sent_len):
      context=sent_len-3
    else:
      context= j

    subCounter=1
    if( true[j] =='O'and pred[j] in ['B-MISC', 'I-MISC']):
      print("{:20} {:10} {:10} [{}]          {:50}".format(word[j],true[j],pred[j], i, " ".join(word[context-window: context+window])))
      count+=1

      print("-------------------------------------------------------------------------")
    j+=subCounter
  i+=1
print(count)

Word                 True       Pred       sentence   contetx
التأسلم              B-MISC     O          [2]          العامة كتاب " التأسلم السياسي .                   
السياسي              I-MISC     O          [2]          العامة كتاب " التأسلم السياسي .                   
-------------------------------------------------------------------------
.                    O          O          [2]          السياسي . . جماعة الإخوان المسلمين                
.                    O          O          [2]          السياسي . . جماعة الإخوان المسلمين                
السياسي              I-MISC     O          [2]          السياسي . . جماعة الإخوان المسلمين                
التأسلم              B-MISC     O          [2]          السياسي . . جماعة الإخوان المسلمين                
جماعة                I-MISC     O          [2]          السياسي . . جماعة الإخوان المسلمين                
الإخوان              I-MISC     B-ORG      [2]          السياسي . . جماعة الإخوان المسلمين                
المسلمين

In [ ]:

## features effect
test_pred = model.predict(np.array(X_test), verbose=1)   
pred_labels1 = pred2label(test_pred)
test_labels = pred2label(Y_test)

X_words_te=X_words[834:]
count=0
i=0
added=[]
print("{:20}: ({:8}): {:8}:{}".format("Word", "True", "Pred", "features"))
while(i<np.shape(pred_labels1)[0]):
  j=0
  pred= pred_labels1[i]
  true=test_labels[i]
  word=X_words_te[i]
  features=  pred_labels2[i]
  while(j<sent_len):
    if(pred[j] != features[j] and true[j] != features[j]):
      print("{:20}: ({:8}): {:8} :{}".format(word[j],true[j],pred[j], features[j]))
      count+=1
      #break
    j+=1
  #added.append((Y_Test_Named[i],Y_Pred2[i]))
  i+=1
print(count)

Word                : (True    ): Pred    :features
ونايفة              : (B-LOC   ): B-LOC    :I-LOC
البادية             : (O       ): O        :B-LOC
الجنوبية            : (O       ): O        :I-LOC
البادية             : (O       ): O        :B-LOC
الشمالية            : (O       ): O        :I-LOC
العامة              : (I-ORG   ): I-ORG    :O
جماعة               : (I-MISC  ): O        :B-ORG
الإخوان             : (I-MISC  ): B-ORG    :I-ORG
قناة                : (B-ORG   ): B-ORG    :O
التأسلم             : (O       ): O        :B-MISC
السياسي             : (O       ): O        :I-MISC
الراشدين            : (B-MISC  ): B-PERS   :O
الخليج              : (O       ): I-MISC   :B-LOC
ووترغيت             : (B-MISC  ): B-MISC   :B-LOC
تشانغسين            : (B-MISC  ): B-LOC    :B-PERS
الأمم               : (B-ORG   ): B-ORG    :I-ORG
مجلس                : (O       ): O        :B-ORG
النواب              : (O       ): O        :I-ORG
بالاتحاد            : (O       ): O        :B-ORG
البابا 

In [ ]:


X_words_te=X_words[834:]
count=0
i=0
added=[]
print("{:20}: ({:8}): {:8}:{}".format("Word", "True", "Pred", "features"))
while(i<np.shape(pred_labels1)[0]):
  j=0
  pred= pred_labels1[i]
  true=test_labels[i]
  word=X_words_te[i]
  features=  pred_labels2[i]
  while(j<sent_len):
    if(pred[j] != features[j] and true[j] == features[j]):
      print("{:20}: ({:8}): {:8} :{}".format(word[j],true[j],pred[j], features[j]))
      count+=1
      #break
    j+=1
  #added.append((Y_Test_Named[i],Y_Pred2[i]))
  i+=1
print(count)

Word                : (True    ): Pred    :features
فأميركا             : (B-LOC   ): O        :B-LOC
التأسلم             : (B-MISC  ): O        :B-MISC
حزب                 : (B-ORG   ): O        :B-ORG
التجمع              : (I-ORG   ): O        :I-ORG
بجماعة              : (B-ORG   ): O        :B-ORG
التكفير             : (I-ORG   ): O        :I-ORG
والهجرة             : (I-ORG   ): O        :I-ORG
جماعة               : (B-ORG   ): O        :B-ORG
الجهاد              : (I-ORG   ): O        :I-ORG
والجماعة            : (B-ORG   ): O        :B-ORG
الإسلامية           : (I-ORG   ): O        :I-ORG
حرب                 : (O       ): B-MISC   :O
الإعصار             : (O       ): B-MISC   :O
تشانغسين            : (B-MISC  ): B-LOC    :B-MISC
الدفاع              : (O       ): I-ORG    :O
المدني              : (O       ): I-ORG    :O
الأوسط              : (O       ): I-LOC    :O
وشانغسين            : (B-MISC  ): B-LOC    :B-MISC
لاريجاني            : (B-PERS  ): I-PERS   :B-PERS
كتائب         

In [ ]:
#previous model
def create_model(lstm_units, embedding_dropout, lstm_dropout,activation_function, optimizer,dense_dropout , second_layer, dense_units):

  ##word input as indecies relvent to total number of words
  word_input = Input(shape=(sent_len,emb_size),name='word_input') ##sent_len is sentence length 

  ## word embedding layer
  '''embeddings =Embedding(    
                            n_words,
                            np.shape(embedding_matrix)[1],
                            embeddings_initializer=keras.initializers.Constant(embedding_matrix),
                            trainable=False,
                            )(word_input)'''

  embeddings = word_input

  if (use_features):
    feature_in = Input(shape=(sent_len, max_fea_len,),name='feature_input')
    embeddings = concatenate([word_input, feature_in])


  ## Character embedding
  if (character_embedding  is Char_type.LSTM):
    char_in = Input(shape=(sent_len, char_len,),name='char_input')
    emb_char = TimeDistributed(Embedding(input_dim=n_chars + 2 , output_dim=25, 
                              input_length=char_len))(char_in)
    char_enc = TimeDistributed(LSTM(units=25, return_sequences=False, 
                                    recurrent_dropout=0.5))(emb_char)
    embeddings = concatenate([word_input, char_enc])


  if (character_embedding  is Char_type.CNN):
    char_in=Input(shape=(sent_len,char_len,1,),name='char_input')
    emb_char=TimeDistributed(Embedding(input_dim=n_chars + 2,output_dim=25,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(char_in)
    conv2d_out= TimeDistributed(Conv2D(kernel_size=3, filters=25, padding='same',activation='tanh', strides=1))(emb_char)
    maxpool_out=TimeDistributed(MaxPooling2D(1))(conv2d_out)
    char_enc = TimeDistributed(Flatten())(maxpool_out)
    embeddings = concatenate([word_input, char_enc])

  '''
  ## attention layer 
  embeddings = SeqSelfAttention(
              units=32,
              attention_width=1,
              attention_type=SeqSelfAttention.ATTENTION_TYPE_ADD,
              use_additive_bias=True,
              use_attention_bias=True,
              attention_activation="softmax",
              kernel_regularizer=keras.regularizers.l2((1e-4)*(10**-1)),
              bias_regularizer=keras.regularizers.l2((1e-4)*(10**-1)),
              attention_regularizer_weight=1e-4*(10**-1),  
          )()'''

  ##dropout layer on embedding layer
  model= Dropout(embedding_dropout)(embeddings)


  ## 2 bidirectional LSTM layers 
  model = Bidirectional(LSTM(units=lstm_units, 
                            return_sequences=True, 
                            dropout=lstm_dropout, 
                            recurrent_dropout=lstm_dropout, 
                            kernel_initializer=keras.initializers.he_normal()))(model)
  if (second_layer):
    model = (LSTM(units=lstm_units*2 , 
                            return_sequences=True, 
                            dropout=lstm_dropout,
                            recurrent_dropout=lstm_dropout, 
                            kernel_initializer=keras.initializers.he_normal()))(model)

  '''
  ## self attention layer 
  model = SeqSelfAttention(
              units=32,
              attention_width=1,
              attention_type=SeqSelfAttention.ATTENTION_TYPE_ADD,
              use_additive_bias=True,
              use_attention_bias=True,
              attention_activation="softmax",
              kernel_regularizer=keras.regularizers.l2((1e-4)*(10**-1)),
              bias_regularizer=keras.regularizers.l2((1e-4)*(10**-1)),
              attention_regularizer_weight=1e-4*(10**-1),  
          )(model)'''

  model = TimeDistributed(Dense(dense_units, activation=activation_function))(model)  # previously softmax output layer
  model= Dropout(dense_dropout)(model)

  ##CRF layer
  crf = CRF(9)  
  out = crf(model) 



  if (use_features and character_embedding  in [Char_type.LSTM, Char_type.CNN]):
      inputs = [word_input,char_in, feature_in]
  elif (character_embedding  in [Char_type.LSTM, Char_type.CNN]):
      inputs = [word_input,char_in]
  elif (use_features):
      inputs = [word_input,feature_in]
  else:
    inputs = word_input


  ##compile model 
  model = Model(inputs, out)
  #adam= keras.optimizers.Adam(lr=optimizer, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
  #nadam =keras.optimizers.Nadam(
    #learning_rate=0, beta_1=0.9, beta_2=0.999, epsilon=1e-07)
  model.compile(optimizer=optimizer, loss=crf.loss_function, metrics=[crf.accuracy, 'accuracy'])
  model.summary()


 
  return model

In [ ]:
##================================== =========================== modified Features =========================== =====================================

max_fea_len=11

def embed_pos(value):
  if value in ['part_fut', 'part_neg', 'part_interrog', 'part_verb', 'part_focus', 'part_det','part','part_voc']:
    return [1.0]
  elif value in [ 'pron_rel', 'pron_interrog', 'pron_exclam', 'pron', 'pron_dem']:
    return [2.0]
  elif value in ['abbrev', 'adv_rel', 'adv_interrog', 'adv' ]:
    return [3.0]
  elif value in ['verb', 'verb_pseudo' ]:
    return [4.0]
  elif value in ['conj_sub', 'conj', 'interj']:
    return [5.0]
  elif value in ['noun_prop' ]:
    return [6.0]
  elif value in [ 'noun']:
    return [7.0]
  elif value in [ 'punc' ]:
    return [8.0]
  elif value in [ 'digit' ]:
    return [9.0]
  elif value in [ 'prep']:
    return [10.0]
  elif value in [ 'noun_quant']:
    return [11.0]
  elif value in [ 'adj']:
    return [12.0]
  elif value in ['abbrev' ]:
    return [13.0]
  else:
    return [0.0]

def embed_enc0(value):
  if value in ['0']:
    return [1.0]
  elif value in [ 'na']:
    return [2.0]
  elif value in [  '1p_dobj' , '1p_poss','1s_pron', '1s_poss', '1p_pron','1s_dobj' ]:             
    return [3.0]
  elif value in ['3mp_pron', '3ms_poss', '3ms_pron', '3d_poss','3fp_poss' ,'3fs_poss',
                 '3d_dobj', '3fs_pron', '3mp_poss', '3fp_pron', '3d_pron', '3mp_dobj',
                 '3ms_dobj', '3fp_dobj', '3fs_dobj']:
    return [4.0]
  elif value in [ '2mp_poss', '2fs_poss', '2ms_poss', '2fp_poss', '2ms_pron', '2fs_pron',
                  '2ms_dobj', '2fs_dobj', '2mp_dobj', '2mp_pron',]:
    return [5.0]
  elif value in ['mA_sub', 'mA_rel', 'lA_neg', 'man_rel', 'ma_interrog' ]:
    return [6.0]
  else:
    return [0.0]

def embed_stt(value):
  return one_hot(value, ['i', 'c', 'd', 'na'])

def embed_num(value):
  return one_hot(value, ['p', 's', 'd', 'na']) 

def embed_per(value):
  return one_hot(value, ['1', '2', '3', 'na']) 

def embed_gen(value):
  return one_hot(value, ['f', 'm', 'na']) 

def embed_prc0(value):
  if value in ['0']:
    return [1.0]
  elif value in ['Al_det']:
    return [2.0]
  elif value in ['mA_rel']:
    return [3.0]
  elif value in ['na' ]:
    return [4.0]
  elif value in ['lA_neg' ]:
    return [5.0]
  elif value in ['mA_neg' ]:
    return [6.0]
  else:
    return [0.0] 

def embed_prc1(value):
  if value in ['0']:
    return [1.0]
  elif value in [ 'sa_fut']:
    return [2.0]
  elif value in ['ka_prep' ]:
    return [3.0]
  elif value in ['na' ]:
    return [4.0]
  elif value in ['la_rc']:
    return [5.0]
  elif value in ['la_emph']:
    return [6.0]
  elif value in ['li_prep' ]:
    return [7.0]
  elif value in ['bi_prep' ]:
    return [8.0]
  else:
    return [0.0] 

def embed_prc2(value):
  if value in ['0']:
    return [1.0]
  elif value in [ 'wa_part', 'wa_sub', 'wa_conj', 'fa_conj']:
    return [2.0]
  elif value in ['na' ]:
    return [3.0]
  else:
    return [0.0]

def embed_prc3(value):
  if value in ['0']:
    return [1.0]
  elif value in [ '>a_ques']:
    return [2.0]
  elif value in ['na' ]:
    return [3.0]
  else:
    return [0.0]
    

def one_hot( value, list):
  try:
    hot_idx =float(list.index(value) +1)
    return [hot_idx]
  except:
    return [0.0]


def features( X_words):
  X_fea = []
  i=0
  idx=0
  while i<len(X_words):
    sent_seq = []
    j=0
    sent=X_words[i]
    while (j<len(sent)):
      word=sent[j]
      if (word=="ENDPAD"):
        feature_vector = np.zeros(11)
      else:
        feature_vector=[]
        feature_vector+= embed_pos(data.at[idx,'pos'])
        feature_vector+= embed_enc0(data.at[idx,'enc0'])
        feature_vector+= embed_stt(data.at[idx,'stt'])
        feature_vector+= embed_num(data.at[idx,'num'])
        feature_vector+= embed_gen(data.at[idx,'gen'])
        feature_vector+= embed_prc0(data.at[idx,'prc0'])
        feature_vector+= embed_prc1(data.at[idx,'prc1'])
        feature_vector+= embed_prc2(data.at[idx,'prc2'])
        feature_vector+= embed_prc3(data.at[idx,'prc3'])
        feature_vector+= embed_per(data.at[idx,'per'])

        try:
          if(data.at[idx,"gloss"][0].isupper()):
            feature_vector+=[1.0]
          else:
            feature_vector+=[0.0]
        except:
          feature_vector+=[0.0]
        idx+=1
      sent_seq.append(feature_vector)
      j+=1
      
    X_fea.append(sent_seq)
    i+=1
  return X_fea





In [ ]:
'''## use stem instead of word

data = data.rename(columns={'word': 'dummy', 'stem': 'word'})
data_tr = data_tr.rename(columns={'word': 'dummy', 'stem': 'word'})
data_te = data_te.rename(columns={'word': 'dummy', 'stem': 'word'})

def strip(data):
  for i , row in data.iterrows():
    data.at[i, 'word']=araby.strip_diacritics (  data.at[i, 'word'])
  return data


data_tr=strip(data_tr)
data_te=strip(data_te)
data=strip(data)

!pip install pyarabic
import pyarabic.araby as araby

## use Diac instead of word
data = data.rename(columns={'word': 'dummy', 'diac': 'word'})
data_tr = data_tr.rename(columns={'word': 'dummy', 'diac': 'word'})
data_te = data_te.rename(columns={'word': 'dummy', 'diac': 'word'})

def fill(data):
  for i , row in data.iterrows():
    if(type (data.at[i,'word'])==float):
      data.at[i, 'word']='؛'
  return data



data_tr=fill(data_tr)
data_te=fill(data_te)
data=fill(data)'''

"\ndef strip(data):\n  for i , row in data.iterrows():\n    data.at[i, 'word']=araby.strip_diacritics (  data.at[i, 'word'])\n  return data\n\n\ndata_tr=strip(data_tr)\ndata_te=strip(data_te)\ndata=strip(data)"

In [ ]:
#create model ## test charcter embedidings
def create_model(lstm_units, embedding_dropout, lstm_dropout,activation_function, optimizer, 
                 dense_dropout , second_layer, dense_units, variational_dropout,):

  ##word input 
  '''if (emb_size==1):
    word_input = Input(shape=(sent_len,),name='word_input') ##sent_len is sentence length 
    embeddings=Embedding(input_dim=len(words) + 1, output_dim=20,)(word_input)
  else:
    word_input = Input(shape=(sent_len,emb_size,),name='word_input') ##sent_len is sentence length 
    embeddings=word_input'''

  '''
  ##Feature input 
  if (use_features):
    feature_input = Input(shape=(sent_len,max_fea_len,),name='feature_input')
    embeddings = concatenate([embeddings, feature_input])

  ## Character embedding
  if (character_embedding  is Char_type.LSTM):
    char_input = Input(shape=(sent_len, char_len,),name='char_input')
    emb_char = TimeDistributed(Embedding(input_dim=n_chars + 2 , output_dim=25, 
                              input_length=char_len))(char_input)
    char_enc = TimeDistributed(LSTM(units=25, return_sequences=False, 
                                    recurrent_dropout=0.5))(emb_char)
    embeddings = concatenate([embeddings, char_enc])'''

  if (character_embedding  is Char_type.CNN):
    char_input=Input(shape=(sent_len,char_len,1,),name='char_input')
    emb_char=TimeDistributed(Embedding(input_dim=n_chars + 2,output_dim=16,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(char_input)
    conv2d_out= TimeDistributed(Conv2D(kernel_size=3, filters=50, padding='same',activation='tanh', strides=1))(emb_char)
    maxpool_out=TimeDistributed(MaxPooling2D(1))(conv2d_out)
    char_enc = TimeDistributed(Flatten())(maxpool_out)
    #embeddings = concatenate([embeddings, char_enc])

  ##dropout layer on embedding layer
  #model= Dropout(embedding_dropout, trainable = variational_dropout)(embeddings)
  model= Dropout(embedding_dropout, trainable = variational_dropout)(char_enc)


  ## 2 bidirectional LSTM layers 
  model = Bidirectional(LSTM(units=lstm_units, 
                            return_sequences=True, 
                            dropout=lstm_dropout, 
                            recurrent_dropout=lstm_dropout, 
                            kernel_initializer=keras.initializers.he_normal()))(model)
  if (second_layer):
    model = (LSTM(units=lstm_units*2 , 
                            return_sequences=True, 
                            dropout=lstm_dropout,
                            recurrent_dropout=lstm_dropout, 
                            kernel_initializer=keras.initializers.he_normal()))(model)


  model = TimeDistributed(Dense(dense_units, activation=activation_function))(model)  # previously softmax output layer
  model= Dropout(dense_dropout)(model)
  '''if (use_features):
    feature_input = Input(shape=(sent_len,max_fea_len,),name='feature_input')
    model = concatenate([model, feature_input])'''
  ##CRF layer
  crf = CRF(9)  
  out = crf(model) 

  '''
  if (use_features and character_embedding  in [Char_type.LSTM, Char_type.CNN]):
      inputs = [word_input,char_input, feature_input]
  elif (character_embedding  in [Char_type.LSTM, Char_type.CNN]):
      inputs = [word_input,char_input]
  elif (use_features):
      inputs = [word_input,feature_input]
  else:
    inputs = word_input'''

  ##compile model 
  #model = Model(inputs, out)
  model = Model(char_input, out)

  model.compile(optimizer=optimizer, loss=crf.loss_function, metrics=[crf.accuracy, "accuracy"])
  model.summary()


 
  return model


def train_model(model, batch, epochs,validation_split, callbacks_list):
  if (use_features and character_embedding  is Char_type.LSTM ):
    history = model.fit([np.array(X_train), np.array(X_char_tr), np.array(X_fea_tr)],
                        np.array(Y_train),
                        batch_size=batch,
                        epochs=epochs,
                        validation_split=validation_split,
                        verbose=1,
                        callbacks=callbacks_list)
  elif (character_embedding is Char_type.LSTM):
    history = model.fit([np.array(X_train), np.array(X_char_tr)],
                        np.array(Y_train),
                        batch_size=batch,
                        epochs=epochs,
                        validation_split=validation_split,
                        verbose=1,
                        callbacks=callbacks_list)
  elif (use_features and character_embedding  is Char_type.CNN ):
    history = model.fit([np.array(X_train), np.array(X_char_tr).reshape(len(X_char_tr), sent_len, char_len, 1), np.array(X_fea_tr)],
                        np.array(Y_train),
                        batch_size=batch,
                        epochs=epochs,
                        validation_split=validation_split,
                        verbose=1,
                        callbacks=callbacks_list)
  elif (character_embedding is Char_type.CNN):
    #history = model.fit([np.array(X_train), np.array(X_char_tr).reshape(len(X_char_tr), sent_len, char_len, 1)],
    history = model.fit( np.array(X_char_tr).reshape(len(X_char_tr), sent_len, char_len, 1),

                        np.array(Y_train),
                        batch_size=batch,
                        epochs=epochs,
                        validation_split=validation_split,
                        verbose=1,
                        callbacks=callbacks_list)
    
  elif (use_features):
    history = model.fit([np.array(X_train), np.array(X_fea_tr)],
                        np.array(Y_train),
                        batch_size=batch,
                        epochs=epochs,
                        validation_split=validation_split,
                        verbose=1,
                        callbacks=callbacks_list)
  else:

    history = model.fit(np.array(X_train),
                        np.array(Y_train),
                        batch_size=batch,
                        epochs=epochs,
                        validation_split=validation_split,
                        verbose=1,
                        callbacks=callbacks_list)
    
def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i])
        out.append(out_i)
    return out

def eval_model(model):
  if (use_features and character_embedding  is Char_type.LSTM):
    test_pred = model.predict([np.array(X_test), np.array(X_char_te), np.array(X_fea_te) ], verbose=1)   

  elif (character_embedding  is Char_type.LSTM):
    test_pred = model.predict([np.array(X_test), np.array(X_char_te) ], verbose=1)

  elif (use_features and character_embedding  is Char_type.CNN):
    test_pred = model.predict([np.array(X_test), np.array(X_char_te).reshape(len(X_char_te), sent_len, char_len, 1), np.array(X_fea_te) ], verbose=1)   

  elif (character_embedding  is  Char_type.CNN):
    #test_pred = model.predict([np.array(X_test), np.array(X_char_te).reshape(len(X_char_te), sent_len, char_len, 1) ], verbose=1)   
    test_pred = model.predict(np.array(X_char_te).reshape(len(X_char_te), sent_len, char_len, 1) , verbose=1)   

  elif (use_features):
    test_pred = model.predict([np.array(X_test), np.array(X_fea_te) ], verbose=1)   

  else:
    test_pred = model.predict(np.array(X_test), verbose=1)   
  pred_labels = pred2label(test_pred)
  test_labels = pred2label(Y_test)
  results=[]
  results+=[round(precision_score(test_labels, pred_labels)*100, 2)]
  results+=[round(recall_score(test_labels, pred_labels)*100, 2)]
  results+=[round(f1_score(test_labels, pred_labels)*100, 2)]
  return(results)
  #classification_report(test_labels, pred_labels)


X_embeddings= X_char
embedding_name='char'
random_split=False
use_features=False
character_embedding =Char_type("CNN")
for b in [ ""]:
  if b=="":
    X_char, n_chars=char(X_words)
  if b=="all":
    X_char, n_chars =char1(X_words)

if (use_features and character_embedding  in [Char_type.LSTM, Char_type.CNN]):
  X_train, X_test, Y_train, Y_test, X_char_tr, X_char_te, X_fea_tr, X_fea_te, emb_size=split(X_embeddings)

elif (use_features):
  X_train, X_test, Y_train, Y_test, X_fea_tr, X_fea_te, emb_size=split(X_embeddings)

elif (character_embedding  in [Char_type.LSTM, Char_type.CNN]):
  X_train, X_test, Y_train, Y_test, X_char_tr, X_char_te, emb_size=split(X_embeddings)

else:
  X_train, X_test, Y_train, Y_test, emb_size=split(X_embeddings)


#model= load_model("w.h5", "params.json")
#eval_model(model)

lstm_units=500
activation_function="relu"
embedding_dropout=0.3
lstm_dropout=0.5
optimizer="nadam"
dense_dropout=0.3
second_layer=False
dense_units=100
val=0.0
batch_size=8
epochs=1
variational_dropout= False

for i in [0]:

  df1= experiment(40, False, "CharCNN_50", epochs) ## experiment ( number of runs, repeate runs or increase epochs , name of csv file)


In [ ]:
## first x digits of arabert, it lead to the same performance
X_arabert1 = [[[round(num, 4) for num in word]for word in sent] for sent in X_arabert]
X_arabert1 =np.array(X_arabert1)


In [ ]:
`#fine tune arabertlargev02 with features_2019 and CNN, sent_len = 256
#for lstm_units in [50, 100, 150, 200, 250, 300, 350, 400, 450, 500(), 550, 600, 650, 700, 750, 800, 750, 900]
#for optimizer in ["adam", "nadam", "adagrad", "adamax", "RMSprop", "adadelta"]
#for activation_function in ["relu", "tanh", "softplus", "linear", "softsign", "hard_sigmoid"]:
#for embedding_dropout in [0, 0.1, 0.2, 0.3, 0.4, 0.5]:
#for lstm_dropout in [0, 0.1, 0.2, 0.3, 0.4, 0.5]:
#for dense_dropout in [0, 0.1, 0.2, 0.3, 0.4, 0.5]:
#for dense_units in [10,32,64,100,150,200, 500]:
for val in [0, 0.05, 0.11111111, 0.15, 0.2, 0.25 ]:
#for batch_size in [8, 16, 32, 64]:


#sentence length fix length 
#epochs


In [ ]:
##================================== =========================== Features =========================== =====================================
## values of attributes grouped

max_fea_len=60
'''
pos_values= list(set(data['pos'].values))
cas_values = list(set(data["cas"].values))
enc0_values = list(set(data["enc0"].values))
stt_values = list(set(data["stt"].values))
num_values = list(set(data["num"].values))
gen_values = list(set(data["gen"].values))
vox_values = list(set(data["vox"].values))
asp_values = list(set(data["asp"].values))
per_values = list(set(data["per"].values))
prc0_values = list(set(data["prc0"].values))
prc1_values = list(set(data["prc1"].values))
prc2_values = list(set(data["prc2"].values))
prc3_values = list(set(data["prc3"].values))
mod_values = list(set(data["mod"].values))'''

def embed_pos(value):
  if value in ['part_fut', 'part_neg', 'part_interrog', 'part_verb', 'part_focus', 'part_det','part','part_voc']:
    hot_idx=0
  elif value in [ 'pron_rel', 'pron_interrog', 'pron_exclam', 'pron', 'pron_dem']:
    hot_idx=1
  elif value in ['abbrev', 'adv_rel', 'adv_interrog', 'adv' ]:
    hot_idx=2
  elif value in ['verb', 'verb_pseudo' ]:
    hot_idx=3
  elif value in ['conj_sub', 'conj', 'interj']:
    hot_idx=4
  elif value in ['noun_prop' ]:
    hot_idx=5
  elif value in [ 'noun']:
    hot_idx=6
  elif value in [ 'punc' ]:
    hot_idx=7
  elif value in [ 'digit' ]:
    hot_idx=8
  elif value in [ 'prep']:
    hot_idx=9
  elif value in [ 'noun_quant']:
    hot_idx=10
  elif value in [ 'adj']:
    hot_idx=11
  elif value in ['abbrev' ]:
    hot_idx=12
  else:
    return np.zeros(13).tolist()
  return one_hot(hot_idx, 13 )

def embed_enc0(value):
  if value in ['0']:
    hot_idx=0
  elif value in [ 'na']:
    hot_idx=1
  elif value in [  '1p_dobj' , '1p_poss','1s_pron', '1s_poss', '1p_pron','1s_dobj' ]:             
    hot_idx=2
  elif value in ['3mp_pron', '3ms_poss', '3ms_pron', '3d_poss','3fp_poss' ,'3fs_poss',
                 '3d_dobj', '3fs_pron', '3mp_poss', '3fp_pron', '3d_pron', '3mp_dobj',
                 '3ms_dobj', '3fp_dobj', '3fs_dobj']:
    hot_idx=3
  elif value in [ '2mp_poss', '2fs_poss', '2ms_poss', '2fp_poss', '2ms_pron', '2fs_pron',
                  '2ms_dobj', '2fs_dobj', '2mp_dobj', '2mp_pron',]:
    hot_idx=4
  elif value in ['mA_sub', 'mA_rel', 'lA_neg', 'man_rel', 'ma_interrog' ]:
    hot_idx=5
  else:
    return np.zeros(6).tolist()
  return one_hot(hot_idx, 6)

def embed_stt(value):
  out=np.zeros(4)
  idx= ['i', 'c', 'd', 'na'].index(value)
  out[idx]=1
  return out.tolist()

def embed_num(value):
  out=np.zeros(4)
  idx= ['p', 's', 'd', 'na'].index(value)
  out[idx]=1
  return out.tolist()

def embed_per(value):
  out=np.zeros(4)
  idx= ['1', '2', '3', 'na'].index(value)
  out[idx]=1
  return out.tolist()

def embed_gen(value):
  out=np.zeros(3)
  idx= ['f', 'm', 'na'].index(value)
  out[idx]=1
  return out.tolist()
  return one_hot(value, ) 

def embed_prc0(value):
  if value in ['0']:
    hot_idx=0
  elif value in ['Al_det']:
    hot_idx=1
  elif value in ['mA_rel']:
    hot_idx=2
  elif value in ['na' ]:
    hot_idx=3
  elif value in ['lA_neg' ]:
    hot_idx=4
  elif value in ['mA_neg' ]:
    hot_idx=5
  else:
    return np.zeros(6).tolist()
  return one_hot(hot_idx, 6)

def embed_prc1(value):
  if value in ['0']:
    hot_idx=0
  elif value in [ 'sa_fut']:
    hot_idx=1
  elif value in ['ka_prep' ]:
    hot_idx=2
  elif value in ['na' ]:
    hot_idx=3
  elif value in ['la_rc']:
    hot_idx=4
  elif value in ['la_emph']:
    hot_idx=5
  elif value in ['li_prep' ]:
    hot_idx=6
  elif value in ['bi_prep' ]:
    hot_idx=7
  else:
    return np.zeros(8).tolist()
  return one_hot(hot_idx, 8)

def embed_prc2(value):
  if value in ['0']:
    hot_idx=0
  elif value in [ 'wa_part', 'wa_sub', 'wa_conj', 'fa_conj']:
    hot_idx=1
  elif value in ['na' ]:
    hot_idx=2
  else:
    return np.zeros(3).tolist()
  return one_hot(hot_idx, 3)

def embed_prc3(value):
  if value in ['0']:
    hot_idx= 0
  elif value in [ '>a_ques']:
    hot_idx= 1
  elif value in ['na' ]:
    hot_idx= 2
  else:
    return np.zeros(3).tolist()
  return one_hot(hot_idx, 3)
    

def embed_cas(value):
  if value in ['na']:
    hot_idx= 0
  elif value in ['n']:
    hot_idx= 1
  elif value in ['u']:
    hot_idx= 2
  elif value in ['a']:
    hot_idx= 3
  elif value in ['g']:
    hot_idx= 4
  else:
    return np.zeros(5).tolist()
  return one_hot(hot_idx, 5)

def one_hot( value, size):
  x=np.zeros(size)
  x[value]=1
  return x.tolist()


def features( X_words):
  X_fea = []
  i=0
  idx=0
  while i<len(X_words):
    sent_seq = []
    j=0
    sent=X_words[i]
    while (j<len(sent)):
      word=sent[j]
      if (word=="ENDPAD"):
        feature_vector = np.zeros(max_fea_len)
      else:
        feature_vector=[]
        feature_vector+= embed_pos(data.at[idx,'pos'])
        feature_vector+= embed_enc0(data.at[idx,'enc0'])
        feature_vector+= embed_stt(data.at[idx,'stt'])
        feature_vector+= embed_num(data.at[idx,'num'])
        feature_vector+= embed_gen(data.at[idx,'gen'])
        feature_vector+= embed_prc0(data.at[idx,'prc0'])
        feature_vector+= embed_prc1(data.at[idx,'prc1'])
        feature_vector+= embed_prc2(data.at[idx,'prc2'])
        feature_vector+= embed_prc3(data.at[idx,'prc3'])
        feature_vector+= embed_per(data.at[idx,'per'])
        feature_vector+= embed_cas(data.at[idx, 'cas'])
        try:
          if(data.at[idx,"gloss"][0].isupper()):
            feature_vector+=[1.0]
          else:
            feature_vector+=[0.0]
        except:
          feature_vector+=[0.0]
        idx+=1
      sent_seq.append(feature_vector)
      j+=1
      
    X_fea.append(sent_seq)
    i+=1
  
  print(np.shape(X_fea))
  return X_fea




In [ ]:
max_fea_len=20

def features0( X_words):
  X_fea = []
  i=0
  idx=0
  while i<len(X_words):
    sent_seq = []
    j=0
    sent=X_words[i]
    while (j<len(sent)):
      word=sent[j]
      if (word=="ENDPAD"):
        feature_vector = np.zeros(max_fea_len)
      else:
        feature_vector=[]
        feature_vector+= embed_pos(data.at[idx,'pos'])
        feature_vector+= embed_enc0(data.at[idx,'enc0'])
        #feature_vector+= embed_stt(data.at[idx,'stt'])
        #feature_vector+= embed_num(data.at[idx,'num'])
        #feature_vector+= embed_gen(data.at[idx,'gen'])
        #feature_vector+= embed_prc0(data.at[idx,'prc0'])
        #feature_vector+= embed_prc1(data.at[idx,'prc1'])
        #feature_vector+= embed_prc2(data.at[idx,'prc2'])
        #feature_vector+= embed_prc3(data.at[idx,'prc3'])
        #feature_vector+= embed_per(data.at[idx,'per'])
        #feature_vector+= embed_cas(data.at[idx, 'cas'])
        try:
          if(data.at[idx,"gloss"][0].isupper()):
            feature_vector+=[1.0]
          else:
            feature_vector+=[0.0]
        except:
          feature_vector+=[0.0]
        idx+=1
      sent_seq.append(feature_vector)
      j+=1
      
    X_fea.append(sent_seq)
    i+=1
  
  print(np.shape(X_fea))
  return X_fea


In [ ]:
##================================== =========================== Features Separated =========================== =====================================

##================================== =========================== Quote Features =========================== =====================================


max_fea_len=2
quote1 =['«', '"', '(']
quote2 =['»', '"', ')']







def features_quo( X_words):
  X_fea = []
  i=0
  idx=0
  add_quote = False
  quote_count = 0
  quote_index=-1
  even_flag=True, True, True ##to assure the second quote is not considered
  while i<len(X_words):
    sent_seq = []
    j=0
    sent=X_words[i]
    while (j<len(sent)):
      word=sent[j]
      if (word=="ENDPAD"):
        feature_vector = np.zeros(max_fea_len)
      else:
        feature_vector=[]
        idx+=1
        ##quote feature
        if add_quote :   ## add values
          feature_vector+= [quote_index+1]
          feature_vector+= [quote_count]
          quote_count -= 1
        else:
          feature_vector+=[0,0]

        if not add_quote and word in quote1: # check if words are between quotes
          quote_index= quote1.index(word)
          if (even_flag or word!='"'):
            for k in [1,2,3,4]:
              try:
                if sent[i+k] == quote2[quote_index]:
                  add_quote= True
                  quote_count=4
                  break
              except:
                add_quote= False
          if (word =='"'):  #to avoid the second quote
            even_flag= not even_flag
             
        if (quote_count ==1): #stop adding values
          add_quote = False
      sent_seq.append(feature_vector)
      j+=1
      
    X_fea.append(sent_seq)
    i+=1
  
  print(np.shape(X_fea))
  return X_fea



##================================== =========================== Language analysis Feature =========================== =====================================


max_fea_len=91
def features_analysis( X_words):
  X_fea = []
  i=0
  idx=0
  while i<len(X_words):
    sent_seq = []
    j=0
    sent=X_words[i]
    while (j<len(sent)):
      word=sent[j]
      if (word=="ENDPAD"):
        feature_vector = np.zeros(max_fea_len)
      else:
        feature_vector=[]
        feature_vector+= one_hot1(data.at[idx,'enc0'], enc0_values)
        feature_vector+= one_hot1(data.at[idx,'stt'], stt_values)
        feature_vector+= one_hot1(data.at[idx,'num'], num_values)
        feature_vector+= one_hot1(data.at[idx,'gen'], gen_values)
        feature_vector+= one_hot1(data.at[idx,'prc0'], prc0_values)
        feature_vector+= one_hot1(data.at[idx,'prc1'], prc1_values)
        feature_vector+= one_hot1(data.at[idx,'prc2'], prc2_values)
        feature_vector+= one_hot1(data.at[idx,'prc3'], prc3_values)
        feature_vector+= one_hot1(data.at[idx,'per'], per_values)
        feature_vector+= one_hot1(data.at[idx,'vox'], vox_values)
        feature_vector+= one_hot1(data.at[idx,'asp'], asp_values)
        feature_vector+= one_hot1(data.at[idx, 'cas'], cas_values)
        feature_vector+= one_hot1(data.at[idx, 'mod'], mod_values)
        idx+=1
      sent_seq.append(feature_vector)
      j+=1
      
    X_fea.append(sent_seq)
    i+=1
  
  print(np.shape(X_fea))
  return X_fea



def one_hot1( value, list):
  x=np.zeros(len(list))
  idx= list.index(value)
  x[idx]=1
  return x.tolist()

pos_values= ['pron_interrog', 'part_fut', 'punc', 'part', 'noun', 'adv', 'interj', 'verb', 'conj_sub', 'adv_interrog', 'noun_prop', 'verb_pseudo', 'part_focus', 'part_interrog', 'conj', 'adj', 'part_voc', 'digit', 'abbrev', 'pron', 'pron_rel', 'part_det', 'part_neg', 'part_verb', 'prep', 'pron_dem', 'adv_rel', 'pron_exclam', 'noun_quant']
cas_values= ['na', 'n', 'u', 'a', 'g']
enc0_values= ['3fp_poss', '3fp_pron', '2mp_poss', 'mA_sub', '3mp_pron', '3ms_dobj', '1s_dobj', '2ms_poss', '1s_pron', '3d_dobj', '1p_dobj', '2ms_pron', '3mp_dobj', '1p_poss', 'ma_interrog', '2fs_pron', '2mp_dobj', '2ms_dobj', 'man_rel', '3ms_poss', '3mp_poss', '3d_poss', '3fs_dobj', '3fs_poss', '2fp_poss', '1p_pron', 'mA_rel', 'lA_neg', '3d_pron', 'na', '2fs_poss', '3ms_pron', '1s_poss', '2fs_dobj', '0', '3fp_dobj', '3fs_pron', '2mp_pron']
stt_values= ['c', 'i', 'd', 'na']
num_values= ['d', 'p', 'na', 's']
gen_values= ['f', 'm', 'na']
vox_values= ['a', 'p', 'na']
asp_values= ['c', 'i', 'p', 'na']
per_values= ['1', '3', 'na', '2']
prc0_values= ['mA_rel', 'mA_neg', 'lA_neg', 'na', 'Al_det', '0']
prc1_values= ['na', 'ka_prep', 'bi_prep', 'li_prep', 'la_emph', '0', 'la_rc', 'sa_fut']
prc2_values = ['na', 'wa_sub', 'wa_conj', 'wa_part', 'fa_conj', '0']
prc3_values= ['>a_ques', '0', 'na']
mod_values= ['i', 'na', 'u']

##================================== =========================== POS Feature =========================== =====================================


max_fea_len=29
def features_pos( X_words):
  X_fea = []
  i=0
  idx=0
  while i<len(X_words):
    sent_seq = []
    j=0
    sent=X_words[i]
    while (j<len(sent)):
      word=sent[j]
      if (word=="ENDPAD"):
        feature_vector = np.zeros(max_fea_len)
      else:
        feature_vector=[]
        feature_vector+= one_hot1(data.at[idx,'pos'], pos_values)
        idx+=1
      sent_seq.append(feature_vector)
      j+=1
      
    X_fea.append(sent_seq)
    i+=1
  
  print(np.shape(X_fea))
  return X_fea



def one_hot1( value, list):
  x=np.zeros(len(list))
  idx= list.index(value)
  x[idx]=1
  return x.tolist()

pos_values= ['pron_interrog', 'part_fut', 'punc', 'part', 'noun', 'adv', 'interj', 'verb', 'conj_sub', 'adv_interrog', 'noun_prop', 'verb_pseudo', 'part_focus', 'part_interrog', 'conj', 'adj', 'part_voc', 'digit', 'abbrev', 'pron', 'pron_rel', 'part_det', 'part_neg', 'part_verb', 'prep', 'pron_dem', 'adv_rel', 'pron_exclam', 'noun_quant']


##================================== =========================== Capitalization Feature =========================== =====================================


max_fea_len=1
def features_cap( X_words):
  X_fea = []
  i=0
  idx=0
  while i<len(X_words):
    sent_seq = []
    j=0
    sent=X_words[i]
    while (j<len(sent)):
      word=sent[j]
      if (word=="ENDPAD"):
        feature_vector = np.zeros(max_fea_len)
      else:
        feature_vector=[]
        try:
          if(data.at[idx,"gloss"][0].isupper()):
            feature_vector+=[1.0]
          else:
            feature_vector+=[0.0]
        except:
          feature_vector+=[0.0]
        idx+=1
      sent_seq.append(feature_vector)
      j+=1
      
    X_fea.append(sent_seq)
    i+=1
  
  print(np.shape(X_fea))
  return X_fea


#max_fea_len = 29    ## 123 for all, 1 for cap, 2 for quote, 29 for pos, 91 for analysis
#X_fea=features_pos(X_words)



In [ ]:
def fill_gaps(test_labels): ## fill gaps in entities
  count=0
  i=0
  while i<len(test_labels):
    j=0
    while j < len(test_labels[i]): 
      if 'I-' in test_labels[i][j] and test_labels[i][j-1] =='O':
        test_labels[i][j-1] = test_labels[i][j]
        count+=1
      j+=1
    i+=1
  if (count!= 0):
    test_labels =fill_gaps(test_labels)
  return test_labels

In [ ]:
###================ evaluation method ===============
#this method evaluate all NEs, hocwver results are different by a small margin

def evaluate(test_labels, pred_labels):
  i=0 # loop on sentences
  TP=[0,0,0,0]
  FN=[0,0,0,0]
  FP=[0,0,0,0]
  FPC=[0,0,0,0] # FPC is entities that are not FP, they should equal TP but they are not as there is a condition that is not handeled
               #to do : add the difference between FPC and TP to FP.

  classes = ['PERS', 'LOC', 'ORG', 'MISC']
  i=0
  while i < len(test_labels):
    j=0 # loop on tokens
    while j < len (test_labels[i]):
      if('B-'in test_labels[i][j]):
        class_name= test_labels[i][j] [2:]
        while (True):
          if (test_labels[i][j] != pred_labels[i][j]):
            j+=1
            FN[classes.index(class_name)]+=1
            break;
          elif class_name not in test_labels[i][j+1] or 'B-' in test_labels[i][j+1] :
            if class_name in pred_labels[i][j+1] and 'B-' not in pred_labels[i][j+1]: 
              FN[classes.index(class_name)]+=1
            else:
              TP[classes.index(class_name)]+=1
            j+=1
            break;
          else:
            j+=1
      else:
        j+=1
    i+=1


  i=0
  while i < len(pred_labels):
    j=0 # loop on tokens
    while j < len (pred_labels[i]):
      if('B-'in pred_labels[i][j]):
        class_name= pred_labels[i][j] [2:]
        while (True):
          if (pred_labels[i][j] != test_labels[i][j]):
            j+=1
            FP[classes.index(class_name)]+=1
            break;
          elif class_name not in pred_labels[i][j+1] or 'B-' in pred_labels[i][j+1]:
            j+=1
            FPC[classes.index(class_name)]+=1

            break;
          else:
            j+=1
      else:
        j+=1
    i+=1
  print('TP: ',TP)
  print('FN: ',FN)
  print('FP: ',FP)
  print('FPC: ',FPC)

  #recall=[sum(x) for x in zip(TP, FN)]




In [ ]:

def evaluate_one_sent(test_labels, pred_labels):
  TP=[0,0,0,0]
  FN=[0,0,0,0]
  FP=[0,0,0,0]
  FPC=[0,0,0,0]

  classes = ['PERS', 'LOC', 'ORG', 'MISC']
  j=0 # loop on tokens
  while j < len (test_labels):
    if('B-'in test_labels[j]):
      class_name= test_labels[j] [2:]
      while (True):
        if (test_labels[j] != pred_labels[j]):
          j+=1
          FN[classes.index(class_name)]+=1
          break;
        elif class_name not in test_labels[j+1] or 'B-' in test_labels[j+1]  :
          if class_name in pred_labels[j+1] and 'B-' not in pred_labels[j+1]: 
            FN[classes.index(class_name)]+=1
          else:
            TP[classes.index(class_name)]+=1
          j+=1
          break;
        else:
          j+=1
    else:
      j+=1

  j=0 # loop on tokens
  while j < len (pred_labels):
    if('B-'in pred_labels[j]):
      class_name= pred_labels[j] [2:]
      while (True):
        if (pred_labels[j] != test_labels[j]):
          j+=1
          FP[classes.index(class_name)]+=1
          break;
        elif class_name not in pred_labels[j+1] or 'B-' in pred_labels[j+1]  :
          j+=1
          FPC[classes.index(class_name)]+=1
          break;
        else:
          j+=1
    else:
      j+=1

  return TP, FN, FP, FPC





'''
TP, FN =evaluate_one_sent(test_labels[0][0:36], pred_labels[0][0:36])
counted_entities=[sum(x) for x in zip(TP, FN)]
print(counted_entities)

i=0
j=0
while j < len(test_labels[i]) :
  print(test_labels[i][j], pred_labels[i][j])
  j+=1

  actual_entities =[0,0,0,0]
i=0
while i <len(test_labels):
  actual_entities_temp =[0,0,0,0]
  for tag in pred_labels[i]:
      if(tag in ['B-PERS', 'B-LOC', 'B-ORG', 'B-MISC']):
        actual_entities_temp[classes.index(tag[2:])]+=1
  actual_entities=[sum(x) for x in zip(actual_entities, actual_entities_temp)]

  
  i+=1


  i=0
count=0
FP_all=[0,0,0,0]
FPC_all=[0,0,0,0]

while i <len(test_labels):
  
  TP, FN, FP, FPC =evaluate_one_sent(test_labels[i], pred_labels[i])
  counted_entities=[sum(x) for x in zip(FP, FPC)]
  FP_all=[sum(x) for x in zip(FP_all, FP)]
  FPC_all=[sum(x) for x in zip(FPC_all, FPC)]


  actual_entities =[0,0,0,0]
  for tag in pred_labels[i]:
    if(tag in ['B-PERS', 'B-LOC', 'B-ORG', 'B-MISC']):
      actual_entities[classes.index(tag[2:])]+=1

  if (counted_entities !=actual_entities):
    print(counted_entities)
    print(actual_entities) 
    print(i) 
    print('-------------------------')
    count+=1
  i+=1

print(count)
print(FP_all)
print(FPC_all)



'''
